<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/Rizwan_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# ================================
# One-cell Colab script:
# Recipe→Property predictor + Forward optimization to hit target specs
# Works with stress–strain columns like:
#   Strain(%)_Cel10_e, Stress(kPa)_Cel10_e, ...
# If you already have true formulation columns (e.g., monomer%, crosslinker%), set RECIPE_FEATURES below.
# ================================
!pip -q install scikit-learn pandas numpy scipy

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import differential_evolution

# ========= CONFIG (edit these) =========
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"  # <-- upload or mount Drive and set the path

# If you have explicit formulation columns in your sheet (e.g., 'monomer_wt%', 'crosslinker_wt%', ...), put them here:
RECIPE_FEATURES: list = []   # e.g., ["monomer_wt%", "crosslinker_wt%", "initiator_wt%", "temp_C", "cure_min"]

# Choose which properties to target. Provide *any subset*; others will be ignored in the objective.
# Common property keys computed by this script:
#   "E0_5_kPa","E5_10_kPa","TanE10_kPa","Yield_strain_frac","Yield_stress_kPa",
#   "Resilience_kJ_m3","UTS_kPa","Strain_UTS_frac","Fracture_strain_frac",
#   "Fracture_stress_kPa","Toughness_kJ_m3","Stress@5%_kPa","Stress@10%_kPa",
#   "Stress@15%_kPa","Stress@20%_kPa","Secant_0_15_kPa"
TARGETS: Dict[str, float] = {
    "UTS_kPa": 90.0,            # example: optimize for UTS only
    # "Toughness_kJ_m3": 7.0,   # add more targets if you want
    # "Strain_UTS_frac": 0.16,
}

# Optimization settings (tweak if needed)
DE_MAXITER = 180
DE_POPSIZE = 18
# ======================================

# ---------- Helpers for stress–strain parsing ----------
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Your data is like 0, 500, 1000 ⇒ 0.0%, 5.0%, 10.0%. Convert to strain fraction.
    return (eps_raw.astype(float) / 100.0) / 100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float) -> Optional[float]:
    if b <= a: return None
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init * (xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0 * (x1 - x0) / (y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float) -> Optional[float]:
    x = p / 100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_recipe_from_label(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

# ---------- Load & derive properties ----------
df = pd.read_excel(DATA_PATH)
labels_strain, labels_stress, labels = find_pairs(df)

records = []
for lab in labels:
    eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
    sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
    mask = np.isfinite(eps_raw) & np.isfinite(sig_raw)
    eps_raw, sig_raw = eps_raw[mask], sig_raw[mask]
    if len(eps_raw) < 3: continue

    eps = to_fraction(eps_raw)
    eps, sig = ensure_sorted(eps, sig_raw)
    f, (lo, hi) = interp_curve(eps, sig)

    # Moduli
    E0_5  = (linear_fit_window(eps, sig, 0.00, 0.05) or (np.nan, np.nan))[0]
    E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
    TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan

    # Yield via 0.2% offset, resilience
    eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
    resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)

    # UTS & fracture
    uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
    frac_strain = float(eps.max()); frac_stress = float(sig[-1])
    toughness = integrate_toughness(eps, sig, None)

    # Probes
    s5  = stress_at(f, 5, lo, hi)
    s10 = stress_at(f, 10, lo, hi)
    s15 = stress_at(f, 15, lo, hi)
    s20 = stress_at(f, 20, lo, hi)
    Sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan

    rec = {
        "label": lab,
        "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
        "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
        "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
        "Resilience_kJ_m3":  resilience,
        "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
        "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
        "Toughness_kJ_m3": toughness,
        "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
        "Secant_0_15_kPa": Sec_0_15,
    }

    # Recipe features
    if RECIPE_FEATURES:
        # Expect these columns to be present in df (per-row constants or per-label sheet)
        # If your recipe data is in another sheet/table, merge it here on "label".
        for c in RECIPE_FEATURES:
            rec[c] = df[c].iloc[0] if c in df.columns else np.nan
    else:
        mat_type, mat_level = parse_recipe_from_label(lab)
        rec["mat_type"] = mat_type
        rec["mat_level"] = mat_level

    records.append(rec)

props_df = pd.DataFrame(records)

# ---------- Build model dataset ----------
# Choose X (recipe) and Y (properties). If RECIPE_FEATURES empty, use mat_type + mat_level from labels.
if RECIPE_FEATURES:
    Xcols = RECIPE_FEATURES
else:
    Xcols = ["mat_type", "mat_level"]

Ycols_all = [
    "E0_5_kPa","E5_10_kPa","TanE10_kPa","Yield_strain_frac","Yield_stress_kPa",
    "Resilience_kJ_m3","UTS_kPa","Strain_UTS_frac","Fracture_strain_frac",
    "Fracture_stress_kPa","Toughness_kJ_m3","Stress@5%_kPa","Stress@10%_kPa",
    "Stress@15%_kPa","Stress@20%_kPa","Secant_0_15_kPa"
]
# Only keep Y columns that exist (depending on curve coverage)
Ycols = [c for c in Ycols_all if c in props_df.columns]

# Fill NaN values in target columns with the mean of existing values
for col in Ycols:
    if props_df[col].isnull().any():
        mean_val = props_df[col].mean()
        props_df[col].fillna(mean_val, inplace=True)

data = props_df[Xcols + Ycols].dropna()

if len(data) < 3:
    raise RuntimeError("Not enough complete rows to train. Ensure recipe features are set or label parsing works and curves cover the needed ranges.")


X = data[Xcols].copy()
Y = data[Ycols].copy()

# ---------- Model pipeline ----------
transformers = []
if any(X[c].dtype == "O" for c in X.columns):  # has categorical
    cat_cols = [c for c in X.columns if X[c].dtype == "O"]
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))
num_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
if num_cols:
    transformers.append(("num", StandardScaler(), num_cols))
preprocess = ColumnTransformer(transformers) if transformers else "passthrough"

base = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
reg = Pipeline([("prep", preprocess), ("rf", MultiOutputRegressor(base))])

# Train / quick eval
ts = 0.33 if len(X) >= 6 else 0.5 if len(X) >= 4 else 0.0
if ts > 0:
    X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, test_size=ts, random_state=42)
else:
    X_tr, Y_tr = X, Y
    X_te, Y_te = X.iloc[:0], Y.iloc[:0]

reg.fit(X_tr, Y_tr)

if len(X_te) > 0:
    Y_hat = reg.predict(X_te)
    for i, col in enumerate(Y.columns):
        r2 = r2_score(Y_te.iloc[:, i], Y_hat[:, i])
        mae = mean_absolute_error(Y_te.iloc[:, i], Y_hat[:, i])
        print(f"[Test] {col:>18} | R2={r2:6.3f} MAE={mae:8.3f}")
else:
    print("Trained on all rows (dataset small).")

# ---------- Forward optimization ----------
# We only consider the properties listed in TARGETS.
target_keys = [k for k in TARGETS.keys() if k in Y.columns]
if len(target_keys) == 0:
    raise ValueError("None of the TARGETS keys match the computed properties. Check your TARGETS dict.")

target_vec = np.array([TARGETS[k] for k in target_keys], float)

# feature bounds for continuous variables
bounds = []
discrete_enums = {}  # for categorical enumeration
for c in X.columns:
    if np.issubdtype(X[c].dtype, np.number):
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))
    else:
        # categorical -> enumerate later; mark placeholder in bounds to keep indexing aligned
        discrete_enums[c] = sorted(X[c].unique().tolist())
        bounds.append(None)

# Build a template row for prediction
def build_row(vec, cat_choice):
    row = {}
    j = 0
    for i, col in enumerate(X.columns):
        if col in discrete_enums:
            row[col] = cat_choice[col]
        else:
            row[col] = float(vec[j]); j += 1
    return pd.DataFrame([row])

# Loss over selected targets (weighted MSE + MAPE)
Y_slice_idx = [Y.columns.get_loc(k) for k in target_keys]
var = np.var(Y.iloc[:, Y_slice_idx].values, axis=0)
weights = 1.0 / np.where(var <= 1e-12, 1.0, var)

def loss_from_row(dfrow):
    yhat = reg.predict(dfrow)[0]
    ysel = yhat[Y_slice_idx]
    eps = 1e-8
    mse = np.mean(weights * (ysel - target_vec)**2)
    mape = np.mean(np.abs(ysel - target_vec) / (np.abs(target_vec) + eps))
    return 0.5*mse + 0.5*mape, yhat

best = {"loss": np.inf, "row": None, "yhat": None}

# Enumerate categoricals (if any), optimize continuous with differential evolution
cat_product = [{}]
for c, vals in discrete_enums.items():
    new = []
    for base_choice in cat_product:
        for v in vals:
            choice = base_choice.copy(); choice[c] = v
            new.append(choice)
    cat_product = new

if len(cat_product) == 0:
    cat_product = [{}]

for cat_choice in cat_product:
    # collect bounds for this subspace
    cont_bounds = [b for (b, col) in zip(bounds, X.columns) if col not in discrete_enums]

    if len(cont_bounds) == 0:
        # No continuous features — just evaluate categorical combo
        dfrow = build_row([], cat_choice)
        loss, yhat = loss_from_row(dfrow)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
        continue

    def _wrap(x):
        dfrow = build_row(x, cat_choice)
        loss, _ = loss_from_row(dfrow)
        return loss

    result = differential_evolution(_wrap, bounds=cont_bounds, maxiter=DE_MAXITER, popsize=DE_POPSIZE, tol=1e-6, polish=True, seed=42)
    dfrow = build_row(result.x, cat_choice)
    loss, yhat = loss_from_row(dfrow)
    if loss < best["loss"]:
        best.update({"loss": loss, "row": dfrow, "yhat": yhat})

opt_recipe = best["row"]
opt_pred = pd.DataFrame([best["yhat"]], columns=Y.columns)
report = pd.concat({"recipe_opt": opt_recipe.reset_index(drop=True), "predicted_props": opt_pred[target_keys].reset_index(drop=True)}, axis=1)

print("\n=== Optimal recipe (for the specified TARGETS) ===")
display_cols = list(opt_recipe.columns)
print(opt_recipe[display_cols].to_string(index=False))

print("\n=== Predicted properties on targets ===")
print(opt_pred[target_keys].to_string(index=False))

# Save artifacts
props_out = "/content/derived_properties_table.csv"
opt_out = "/content/optimal_recipe_and_predictions.csv"
props_df.to_csv(props_out, index=False)
report.to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

/tmp/ipython-input-3831001079.py:101: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))
/tmp/ipython-input-3831001079.py:199: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  props_df[col].fillna(mean_val, inplace=True)


RuntimeError: Not enough complete rows to train. Ensure recipe features are set or label parsing works and curves cover the needed ranges.

In [4]:
# ---------- Build model dataset (robust to sparse columns / NaNs) ----------
# Choose X (recipe) and Y (properties). If RECIPE_FEATURES empty, use mat_type + mat_level from labels.
if RECIPE_FEATURES:
    Xcols = RECIPE_FEATURES
else:
    Xcols = ["mat_type", "mat_level"]

# Which targets do you want to fit/optimize on?
all_cols_available = set(props_df.columns)
target_keys = [k for k in TARGETS.keys() if k in all_cols_available]

if len(target_keys) == 0:
    raise ValueError(
        f"None of your TARGETS {list(TARGETS.keys())} exist in computed properties. "
        f"Available: {sorted(all_cols_available)}"
    )

# Count how many non-NaN rows exist per requested target
counts = {k: int(props_df[k].notna().sum()) for k in target_keys}
print("Non-NaN rows per requested target:", counts)

# Keep only targets with at least 2 rows (so the model can learn something)
Ycols = [k for k in target_keys if counts[k] >= 2]

# If nothing qualifies, fall back to a single, common property such as UTS_kPa (if present)
if len(Ycols) == 0:
    if "UTS_kPa" in props_df.columns and props_df["UTS_kPa"].notna().sum() >= 2:
        print("No requested target has >=2 rows. Falling back to Y=['UTS_kPa'].")
        Ycols = ["UTS_kPa"]
        target_keys = ["UTS_kPa"]
        TARGETS = {"UTS_kPa": TARGETS.get("UTS_kPa", float(props_df["UTS_kPa"].median()))}
    else:
        # Print helpful diagnostics and stop
        avail_counts = {c: int(props_df[c].notna().sum()) for c in [
            "UTS_kPa","Toughness_kJ_m3","Strain_UTS_frac","E0_5_kPa","Stress@5%_kPa","Stress@10%_kPa"
        ] if c in props_df.columns}
        raise RuntimeError(
            "Not enough complete rows for your selected targets. "
            f"Try targeting a property with more coverage. Coverage snapshot: {avail_counts}"
        )

use_cols = Xcols + Ycols
data = props_df[use_cols].dropna(subset=Ycols)
if len(data) < 2:
    raise RuntimeError(
        f"After filtering to X={Xcols} and Y={Ycols}, "
        f"only {len(data)} complete rows remain. "
        "Remove sparsely available targets or expand your dataset."
    )

X = data[Xcols].copy()
Y = data[Ycols].copy()

# ---------- Model pipeline ----------
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np
import pandas as pd

transformers = []
cat_cols = [c for c in X.columns if X[c].dtype == "O"]
num_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]

if cat_cols:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))
if num_cols:
    transformers.append(("num", StandardScaler(), num_cols))

preprocess = ColumnTransformer(transformers) if transformers else "passthrough"

base = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
reg = Pipeline([("prep", preprocess), ("rf", MultiOutputRegressor(base))])

# Train / quick eval (handles tiny datasets)
ts = 0.33 if len(X) >= 6 else 0.5 if len(X) >= 4 else 0.0
if ts > 0:
    X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, test_size=ts, random_state=42)
else:
    X_tr, Y_tr = X, Y
    X_te, Y_te = X.iloc[:0], Y.iloc[:0]

reg.fit(X_tr, Y_tr)

if len(X_te) > 0:
    Y_hat = reg.predict(X_te)
    for i, col in enumerate(Y.columns):
        r2 = r2_score(Y_te.iloc[:, i], Y_hat[:, i])
        mae = mean_absolute_error(Y_te.iloc[:, i], Y_hat[:, i])
        print(f"[Test] {col:>18} | R2={r2:6.3f} MAE={mae:8.3f}")
else:
    print("Trained on all rows (dataset small).")

# ---------- Forward optimization (only on your selected targets) ----------
# bounds for continuous features; enumerate any categoricals
bounds = []
discrete_enums = {}
for c in X.columns:
    if np.issubdtype(X[c].dtype, np.number):
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))
    else:
        discrete_enums[c] = sorted(X[c].unique().tolist())
        bounds.append(None)

def build_row(vec, cat_choice):
    row = {}
    j = 0
    for i, col in enumerate(X.columns):
        if col in discrete_enums:
            row[col] = cat_choice[col]
        else:
            row[col] = float(vec[j]); j += 1
    return pd.DataFrame([row])

# assemble the target vector in the same order as Ycols
target_vec = np.array([TARGETS[k] for k in Ycols], float)

# weights for loss: inverse variance of Y-train slice to balance scales
var = np.var(Y_tr[Ycols].values, axis=0) if len(Y_tr) > 0 else np.var(Y[Ycols].values, axis=0)
weights = 1.0 / np.where(var <= 1e-12, 1.0, var)

def loss_from_row(dfrow):
    yhat = reg.predict(dfrow)[0]
    # yhat aligns to Ycols order
    eps = 1e-8
    mse = np.mean(weights * (yhat - target_vec)**2)
    mape = np.mean(np.abs(yhat - target_vec) / (np.abs(target_vec) + eps))
    return 0.5*mse + 0.5*mape, yhat

# enumerate categoricals; DE over continuous
from scipy.optimize import differential_evolution
best = {"loss": np.inf, "row": None, "yhat": None}

cat_product = [{}]
for c, vals in discrete_enums.items():
    new = []
    for base_choice in cat_product:
        for v in vals:
            tmp = base_choice.copy(); tmp[c] = v
            new.append(tmp)
    cat_product = new
if len(cat_product) == 0:
    cat_product = [{}]

for cat_choice in cat_product:
    cont_bounds = [b for (b, col) in zip(bounds, X.columns) if col not in discrete_enums]
    if len(cont_bounds) == 0:
        dfrow = build_row([], cat_choice)
        loss, yhat = loss_from_row(dfrow)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
        continue

    def _wrap(x):
        dfrow = build_row(x, cat_choice)
        loss, _ = loss_from_row(dfrow)
        return loss

    result = differential_evolution(_wrap, bounds=cont_bounds, maxiter=DE_MAXITER, popsize=DE_POPSIZE, tol=1e-6, polish=True, seed=42)
    dfrow = build_row(result.x, cat_choice)
    loss, yhat = loss_from_row(dfrow)
    if loss < best["loss"]:
        best.update({"loss": loss, "row": dfrow, "yhat": yhat})

opt_recipe = best["row"]
opt_pred = pd.DataFrame([best["yhat"]], columns=Ycols)

print("\n=== Optimal recipe (for YOUR targets only) ===")
print(opt_recipe.to_string(index=False))
print("\n=== Predicted properties (on those targets) ===")
print(opt_pred.to_string(index=False))


Non-NaN rows per requested target: {'UTS_kPa': 10}
[Test]            UTS_kPa | R2=-0.526 MAE=  13.781

=== Optimal recipe (for YOUR targets only) ===
mat_type  mat_level
     Cel  15.148414

=== Predicted properties (on those targets) ===
  UTS_kPa
82.020833


In [5]:
print(props_df.isna().mean().sort_values())  # fraction of NaNs per column


label                   0.0
E0_5_kPa                0.0
E5_10_kPa               0.0
TanE10_kPa              0.0
Yield_strain_frac       0.0
Yield_stress_kPa        0.0
Resilience_kJ_m3        0.0
UTS_kPa                 0.0
Strain_UTS_frac         0.0
Fracture_strain_frac    0.0
Fracture_stress_kPa     0.0
Toughness_kJ_m3         0.0
Stress@5%_kPa           0.0
Stress@10%_kPa          0.0
Stress@15%_kPa          0.0
Secant_0_15_kPa         0.0
mat_level               0.0
mat_type                0.0
Stress@20%_kPa          1.0
dtype: float64


In [9]:
# ============================
# Single-cell: Composition design for target UTS (and others)
# ============================
!pip -q install numpy pandas scikit-learn scipy

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ========== USER CONFIG ==========
# 1) Stress–strain file (must have paired columns like Strain(%)_X and Stress(kPa)_X, where X is a label)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE of the two):
#    (A) Separate file (CSV or XLSX) with a table that contains 'label' + composition columns
COMPOSITION_PATH = "/content/composition_table.xlsx"   # e.g., "/content/recipe_table.csv" or ""
#    (B) OR a sheet *inside* DATA_PATH:
COMPOSITION_SHEET = ""  # e.g., "composition" if you kept it in the same workbook

# 3) Which columns in your composition table are the actual composition variables you want the optimizer to choose?
#    Examples: wt% columns that sum to ~100, or molar ratios, etc. These must be numeric.
#    If empty, the code will fall back to ('mat_type', 'mat_level') parsed from labels (coarse).
COMPOSITION_COLS: List[str] = [
    # "monomer_wt%", "crosslinker_wt%", "initiator_wt%", "solvent_wt%", "salt_mM"
]

# 4) Sum constraint for composition variables (e.g., sum of wt% must equal 100)
#    Set to None if not applicable. Otherwise set a float like 100.0
COMPOSITION_TOTAL = 100.0  # or None

# 5) Targets (use any subset). Start with UTS only, then add more as needed.
TARGETS: Dict[str, float] = {
    "UTS_kPa": 90.0,
    # "Toughness_kJ_m3": 7.0,
    # "Strain_UTS_frac": 0.16,
}
# =================================

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Your data pattern looks like: [0, 500, 1000, ...] -> 0.0%, 5.0%, 10.0% -> 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

# ======== Derive properties from curves ========
def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3: continue

        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        E0_5  = (linear_fit_window(eps, sig, 0.00, 0.05) or (np.nan, np.nan))[0]
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ======== Load composition table if provided ========
def load_composition() -> pd.DataFrame:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ======== Build modeling table ========
if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    X = df_model[COMPOSITION_COLS].astype(float).copy()
    # Optional: detect if columns are percentages that should sum to ~COMPOSITION_TOTAL
    sum_close = None
    if COMPOSITION_TOTAL is not None:
        s = X[COMPOSITION_COLS].sum(axis=1)
        sum_close = np.median(np.isclose(s, COMPOSITION_TOTAL, rtol=0.01, atol=1e-2))
        print(f"[Info] Median samples match sum≈{COMPOSITION_TOTAL}? -> {bool(sum_close)}")
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    # Encode mat_type as categorical + mat_level numeric
    X = props_df[["mat_type", "mat_level"]].copy()

# Y: ONLY use the targets you care about (robust against NaNs in other properties)
available = set(props_df.columns)
target_keys = [k for k in TARGETS if k in available]
if not target_keys:
    raise ValueError(f"None of your TARGETS {list(TARGETS.keys())} exist in derived properties. Available: {sorted(available)}")
Y = props_df[["label"] + target_keys].dropna(subset=target_keys)

# Align X and Y by label
if MODE == "composition":
    XY = df_model[["label"] + COMPOSITION_COLS + target_keys].dropna(subset=target_keys + COMPOSITION_COLS)
    X = XY[COMPOSITION_COLS].astype(float)
    Y = XY[target_keys].astype(float)
else:
    XY = props_df[["label", "mat_type", "mat_level"] + target_keys].dropna(subset=target_keys)
    X = XY[["mat_type", "mat_level"]].copy()
    Y = XY[target_keys].astype(float)


if len(X) < 2:
    raise RuntimeError("Not enough rows to train. Ensure targets have at least 2 valid samples and composition columns are present (if using composition mode).")

# ======== Model pipeline ========
transformers = []
if MODE == "composition":
    # all numeric
    num_cols = X.columns.tolist()
    preprocess = ColumnTransformer([("num", StandardScaler(), num_cols)])
else:
    # mat_type categorical + mat_level numeric
    cat_cols = ["mat_type"]
    num_cols = ["mat_level"]
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols),
    ])

base = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
reg = Pipeline([("prep", preprocess), ("rf", MultiOutputRegressor(base))])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, test_size=ts, random_state=42)
else:
    X_tr, Y_tr = X, Y
    X_te, Y_te = X.iloc[:0], Y.iloc[:0]

reg.fit(X_tr, Y_tr)

if len(X_te) > 0:
    Y_hat = reg.predict(X_te)
    print("=== Quick holdout ===")
    for i, col in enumerate(Y.columns):
        print(f"{col:>18}  R2={r2_score(Y_te.iloc[:, i], Y_hat[:, i]):6.3f}  MAE={mean_absolute_error(Y_te.iloc[:, i], Y_hat[:, i]):8.3f}")
else:
    print("Trained on all rows (small dataset).")

# ======== Forward design: get composition (or knobs) for TARGETS ========
# Build loss on your targets only
tgt = np.array([TARGETS[k] for k in target_keys], float)

# Separate bounds for numeric and handle categoricals
bounds = []
cont_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
cat_cols = [c for c in X.columns if X[c].dtype == "O"]
discrete_enums = {c: sorted(X[c].unique().tolist()) for c in cat_cols}

for c in cont_cols:
    lo, hi = float(X[c].min()), float(X[c].max())
    span = hi - lo if hi > lo else 1.0
    bounds.append((lo - 0.05*span, hi + 0.05*span))

# Build a template row for prediction
def build_row(cont_vec, cat_choice):
    row = {}
    j = 0
    for col in X.columns:
        if col in cat_cols:
            row[col] = cat_choice[col]
        else:
            row[col] = float(cont_vec[j]); j += 1
    return pd.DataFrame([row], columns=X.columns) # Ensure columns order matches X

# Sum-to-constant constraint (if composition percentages)
constraints = []
if MODE == "composition" and COMPOSITION_TOTAL is not None:
    def sum_eq(cont_vec, cont_cols=cont_cols, total=COMPOSITION_TOTAL):
        # This assumes cont_cols are the composition components that sum
        return float(np.sum(cont_vec) - total)
    constraints.append({"type": "eq", "fun": sum_eq})

# penalty for going outside historical range (soft corners)
if cont_cols:
    mins = X[cont_cols].min().values
    maxs = X[cont_cols].max().values
else:
    mins = np.array([])
    maxs = np.array([])


def objective(cont_vec, cat_choice):
    row = build_row(cont_vec, cat_choice)
    pred = reg.predict(row)[0]
    # Weighted MSE + MAPE on targets
    var = np.var(Y_tr.values, axis=0) if len(Y_tr) else np.var(Y.values, axis=0)
    var = var if var.size == pred.size else np.ones_like(pred)  # safeguard
    w = 1.0 / np.where(var <= 1e-12, 1.0, var)
    eps = 1e-8
    # align only the target indices (Y order == target_keys order)
    # our Y_tr columns equal target_keys in order already
    y_pred_targets = pred[[Y.columns.get_loc(k) for k in target_keys]] # Ensure target columns are selected from prediction
    mse = np.mean(w * (y_pred_targets - tgt)**2)
    mape = np.mean(np.abs(y_pred_targets - tgt)/(np.abs(tgt)+eps))
    base_loss = 0.5*mse + 0.5*mape

    # Soft penalty if outside observed domain (only for continuous features)
    penalty = 0.0
    if cont_cols:
        over_low = np.maximum(0.0, mins - cont_vec)
        over_high = np.maximum(0.0, cont_vec - maxs)
        penalty = 1e-3 * np.sum(over_low**2 + over_high**2)

    return base_loss + penalty


# Initial guess for continuous: median of continuous columns
x0_cont = X[cont_cols].median().values if cont_cols else np.array([])

# Enumerate categoricals; minimize continuous for each category combination
from scipy.optimize import minimize
best = {"loss": np.inf, "row": None, "yhat": None}

cat_product = [{}]
if cat_cols:
    import itertools
    cat_values = [discrete_enums[c] for c in cat_cols]
    cat_product = [{c: v for c, v in zip(cat_cols, combo)} for combo in itertools.product(*cat_values)]


for cat_choice in cat_product:
    if not cont_cols:
        # No continuous features — just evaluate categorical combo
        dfrow = build_row(np.array([]), cat_choice)
        loss, yhat = objective(np.array([]), cat_choice), reg.predict(dfrow)[0] # Evaluate loss with objective directly
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
        continue

    def _wrap(cont_vec):
        return objective(cont_vec, cat_choice)

    result = minimize(
        _wrap, x0_cont,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 1000, "ftol": 1e-9, "disp": False}
    )

    if result.success:
        loss = result.fun
        dfrow = build_row(result.x, cat_choice)
        yhat = reg.predict(dfrow)[0]
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
    else:
        print(f"[Warn] Optimization failed for {cat_choice}: {result.message}")


if best["row"] is None:
     raise RuntimeError("Optimization failed for all combinations.")


opt_recipe = best["row"]
opt_pred = pd.DataFrame([best["yhat"]], columns=Y.columns) # Use Y.columns for full prediction columns
report = pd.concat({"recipe_opt": opt_recipe.reset_index(drop=True), "predicted_targets": opt_pred[target_keys].reset_index(drop=True)}, axis=1) # Report only target columns


print("\n================ RESULTS ================")
if MODE == "composition":
    print("Mode: COMPOSITION → PROPERTIES")
    print("\nRecommended composition:")
    display_cols = COMPOSITION_COLS
else:
    print("Mode: LABEL FALLBACK (no composition provided)")
    print("\nRecommended knobs parsed from label (not true chemistry):")
    display_cols = X.columns.tolist()

print(opt_recipe[display_cols].to_string(index=False))
print("\nPredicted target properties:")
print(report.to_string(index=False))

# Save files
props_out = "/content/_derived_properties_table.csv"
opt_out = "/content/_optimal_recipe_and_predictions.csv"
props_df.to_csv(props_out, index=False)
report.to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# Helpful tip: if you get "not enough rows", print coverage per target:
# print(props_df[target_keys].notna().sum())

/tmp/ipython-input-3718803818.py:103: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.
=== Quick holdout ===
           UTS_kPa  R2=-0.548  MAE=  13.855

================ RESULTS ================
Mode: LABEL FALLBACK (no composition provided)

Recommended knobs parsed from label (not true chemistry):
mat_type  mat_level
     Cel        5.0

Predicted target properties:
recipe_opt           predicted_targets
  mat_type mat_level           UTS_kPa
       Cel       5.0         61.194722

Saved:
  /content/_derived_properties_table.csv
  /content/_optimal_recipe_and_predictions.csv


What you’ll see

If you provided COMPOSITION_COLS and a composition table with label matching the stress–strain labels, you’ll get a recommended chemical composition (respecting a sum=100 constraint if you set COMPOSITION_TOTAL=100.0) that best hits your TARGETS.

If you skip composition, you’ll get recommended coarse knobs (mat_type, mat_level), which is not a full chemistry but still runs.

Common pitfalls (and quick fixes)

Error: not enough rows → Start with TARGETS = {"UTS_kPa": <value>}; UTS is almost always available.

Sum constraint → If your variables are wt%, keep COMPOSITION_TOTAL=100.0. If not percentages, set it to None.

Column names → Make sure COMPOSITION_COLS exactly match your table’s numeric columns and that a label column matches the stress–strain label suffixes.

If you share the names of your actual composition columns (e.g., AAm_wt, BIS_wt, APS_wt, TEMED_wt, Water_wt, NaCl_mM), I can pre-fill COMPOSITION_COLS (and the appropriate COMPOSITION_TOTAL) for you.

1) UTS only

In [12]:
# ============================
# Single-cell: Composition design for a chosen target property (or set)
# ============================
!pip -q install numpy pandas scikit-learn scipy

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ========== USER CONFIG ==========
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE)
COMPOSITION_PATH = ""                 # e.g., "/content/composition_table.xlsx" or "/content/recipe_table.csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet inside DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level')
COMPOSITION_COLS: List[str] = [
    # "monomer_wt%", "crosslinker_wt%", "initiator_wt%", "solvent_wt%", "salt_mM"
]

# 4) If composition columns are wt% (or any total constraint), set sum=constant
COMPOSITION_TOTAL = 100.0   # or None if not applicable

# 5) >>>>> CHANGE THIS FOR EACH TARGET <<<<<
TARGETS: Dict[str, float] = {
    "UTS_kPa": 90.0,    # example default; replace per the presets below
}
# =================================

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3: continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)
        E0_5  = (linear_fit_window(eps, sig, 0.00, 0.05) or (np.nan, np.nan))[0]
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan
        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

def load_composition() -> pd.DataFrame:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# Build X (recipe features) and Y (targets)
if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + list(TARGETS.keys())]
    XY = XY.dropna(subset=COMPOSITION_COLS + list(TARGETS.keys()))
    X = XY[COMPOSITION_COLS].astype(float)
    Y = XY[list(TARGETS.keys())].astype(float)
else:
    MODE = "label_fallback"
    XY = props_df[["label","mat_type","mat_level"] + list(TARGETS.keys())].dropna(subset=list(TARGETS.keys()))
    X = XY[["mat_type","mat_level"]].copy()
    Y = XY[list(TARGETS.keys())].astype(float)

if len(X) < 2:
    raise RuntimeError("Not enough rows to train. Ensure your chosen target has at least 2 valid samples.")

# Pipeline
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])
base = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
reg = Pipeline([("prep", preprocess), ("rf", MultiOutputRegressor(base))])

ts = 0.33 if len(X) >= 6 else 0.0
if ts>0:
    X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, test_size=ts, random_state=42)
else:
    X_tr, Y_tr = X, Y
    X_te, Y_te = X.iloc[:0], Y.iloc[:0]

reg.fit(X_tr, Y_tr)
if len(X_te)>0:
    Y_hat = reg.predict(X_te)
    print("=== Quick holdout ===")
    for i, col in enumerate(Y.columns):
        print(f"{col:>18}  R2={r2_score(Y_te.iloc[:, i], Y_hat[:, i]):6.3f}  MAE={mean_absolute_error(Y_te.iloc[:, i], Y_hat[:, i]):8.3f}")
else:
    print("Trained on all rows (small dataset).")

# Forward design (SLSQP with optional sum constraint)
tgt = np.array([TARGETS[k] for k in Y.columns], float)

# Separate bounds for numeric and handle categoricals
bounds = []
cont_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
cat_cols = [c for c in X.columns if X[c].dtype == "O"]
discrete_enums = {c: sorted(X[c].unique().tolist()) for c in cat_cols}

for c in cont_cols:
    lo, hi = float(X[c].min()), float(X[c].max())
    span = hi - lo if hi > lo else 1.0
    bounds.append((lo - 0.05*span, hi + 0.05*span))

# Build a template row for prediction
def build_row(cont_vec, cat_choice):
    row = {}
    j = 0
    for col in X.columns:
        if col in cat_cols:
            row[col] = cat_choice[col]
        else:
            row[col] = float(cont_vec[j]); j += 1
    return pd.DataFrame([row], columns=X.columns) # Ensure columns order matches X

# Sum-to-constant constraint (if composition percentages)
constraints = []
if MODE == "composition" and COMPOSITION_TOTAL is not None:
    def sum_eq(cont_vec, cont_cols=cont_cols, total=COMPOSITION_TOTAL):
        # This assumes cont_cols are the composition components that sum
        return float(np.sum(cont_vec) - total)
    constraints.append({"type": "eq", "fun": sum_eq})

# penalty for going outside historical range (soft corners)
if cont_cols:
    mins = X[cont_cols].min().values
    maxs = X[cont_cols].max().values
else:
    mins = np.array([])
    maxs = np.array([])


def objective(cont_vec, cat_choice):
    row = build_row(cont_vec, cat_choice)
    pred = reg.predict(row)[0]
    # Weighted MSE + MAPE on targets
    var = np.var(Y_tr.values, axis=0) if len(Y_tr) else np.var(Y.values, axis=0)
    var = var if var.size == pred.size else np.ones_like(pred)  # safeguard
    w = 1.0 / np.where(var <= 1e-12, 1.0, var)
    eps = 1e-8
    # align only the target indices (Y order == target_keys order)
    # our Y_tr columns equal target_keys in order already
    y_pred_targets = pred[[Y.columns.get_loc(k) for k in target_keys]] # Ensure target columns are selected from prediction
    mse = np.mean(w * (y_pred_targets - tgt)**2)
    mape = np.mean(np.abs(y_pred_targets - tgt)/(np.abs(tgt)+eps))
    base_loss = 0.5*mse + 0.5*mape

    # Soft penalty if outside observed domain (only for continuous features)
    penalty = 0.0
    if cont_cols:
        over_low = np.maximum(0.0, mins - cont_vec)
        over_high = np.maximum(0.0, cont_vec - maxs)
        penalty = 1e-3 * np.sum(over_low**2 + over_high**2)

    return base_loss + penalty


# Initial guess for continuous: median of continuous columns
x0_cont = X[cont_cols].median().values if cont_cols else np.array([])

# Enumerate categoricals; minimize continuous for each category combination
from scipy.optimize import minimize
best = {"loss": np.inf, "row": None, "yhat": None}

cat_product = [{}]
if cat_cols:
    import itertools
    cat_values = [discrete_enums[c] for c in cat_cols]
    cat_product = [{c: v for c, v in zip(cat_cols, combo)} for combo in itertools.product(*cat_values)]


for cat_choice in cat_product:
    if not cont_cols:
        # No continuous features — just evaluate categorical combo
        dfrow = build_row(np.array([]), cat_choice)
        loss, yhat = objective(np.array([]), cat_choice), reg.predict(dfrow)[0] # Evaluate loss with objective directly
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
        continue

    def _wrap(cont_vec):
        return objective(cont_vec, cat_choice)

    result = minimize(
        _wrap, x0_cont,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 1000, "ftol": 1e-9, "disp": False}
    )

    if result.success:
        loss = result.fun
        dfrow = build_row(result.x, cat_choice)
        yhat = reg.predict(dfrow)[0]
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
    else:
        print(f"[Warn] Optimization failed for {cat_choice}: {result.message}")


if best["row"] is None:
     raise RuntimeError("Optimization failed for all combinations.")


opt_recipe = best["row"]
opt_pred = pd.DataFrame([best["yhat"]], columns=Y.columns) # Use Y.columns for full prediction columns
report = pd.concat({"recipe_opt": opt_recipe.reset_index(drop=True), "predicted_targets": opt_pred[target_keys].reset_index(drop=True)}, axis=1) # Report only target columns


print("\n================ RESULTS ================")
if MODE == "composition":
    print("Mode: COMPOSITION → PROPERTIES")
    print("\nRecommended composition:")
    display_cols = COMPOSITION_COLS
else:
    print("Mode: LABEL FALLBACK (no composition provided)")
    print("\nRecommended knobs parsed from label (not true chemistry):")
    display_cols = X.columns.tolist()

print(opt_recipe[display_cols].to_string(index=False))
print("\nPredicted target properties:")
print(report.to_string(index=False))

# Save files
props_out = "/content/_derived_properties_table.csv"
opt_out = "/content/_optimal_recipe_and_predictions.csv"
props_df.to_csv(props_out, index=False)
report.to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# Helpful tip: if you get "not enough rows", print coverage per target:
# print(props_df[target_keys].notna().sum())

/tmp/ipython-input-3754016808.py:95: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


=== Quick holdout ===
           UTS_kPa  R2=-0.548  MAE=  13.855

================ RESULTS ================
Mode: LABEL FALLBACK (no composition provided)

Recommended knobs parsed from label (not true chemistry):
mat_type  mat_level
     Cel        5.0

Predicted target properties:
recipe_opt           predicted_targets
  mat_type mat_level           UTS_kPa
       Cel       5.0         61.194722

Saved:
  /content/_derived_properties_table.csv
  /content/_optimal_recipe_and_predictions.csv


Here’s a single, copy-paste Colab cell tailored to optimize Toughness only. It:

Reads your stress–strain file from Google Drive (path you gave)

Derives mechanical properties per curve

Trains a recipe → Toughness predictor

Designs a recipe/composition to hit your target Toughness_kJ_m3

Works with either a true composition table (recommended) or a fallback using mat_type + mat_level parsed from labels

How to use:

Make sure your file exists at the path you gave.

(Optional) If you have a composition table, fill COMPOSITION_PATH (or COMPOSITION_SHEET) and list its numeric columns in COMPOSITION_COLS (e.g., monomer %, crosslinker %, …). Set COMPOSITION_TOTAL=100.0 if they are wt%.

Set your toughness target in TARGET_TOUGHNESS. Run the cel

2) Toughness only

In [16]:
# ============================================
# One-cell: Toughness-only composition design
# ============================================
!pip -q install numpy pandas scikit-learn scipy

# --- Mount Google Drive (needed for the provided path) ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ---------------- USER CONFIG ----------------
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE: separate file OR sheet within DATA_PATH)
COMPOSITION_PATH = ""                 # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx" or ".csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet in DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level')
COMPOSITION_COLS: List[str] = [
    # Example: "AAm_wt", "BIS_wt", "APS_wt", "TEMED_wt", "Water_wt"
]

# 4) If composition cols are wt% (or any total constraint), set the target sum (e.g., 100.0). Else set None.
COMPOSITION_TOTAL = 100.0   # or None

# 5) Toughness target (kJ/m^3). Set your desired value here:
TARGET_TOUGHNESS = 7.0
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Your data pattern looks like: [0, 500, 1000, ...] -> 0.0%, 5.0%, 10.0% -> 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))  # kJ/m^3 if stress in kPa and strain is fraction

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3: continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)
        E0_5  = (linear_fit_window(eps, sig, 0.00, 0.05) or (np.nan, np.nan))[0]
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan
        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ---------- Load composition (optional) ----------
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ---------- Build X (recipe) and Y (toughness) ----------
TARGET_KEY = "Toughness_kJ_m3"
TARGETS = {TARGET_KEY: TARGET_TOUGHNESS} # Ensure TARGETS dict is created

if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + [TARGET_KEY]].dropna(subset=COMPOSITION_COLS + [TARGET_KEY])
    X = XY[COMPOSITION_COLS].astype(float)
    Y = XY[[TARGET_KEY]].astype(float)
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    XY = props_df[["label","mat_type","mat_level", TARGET_KEY]].dropna(subset=[TARGET_KEY])
    X = XY[["mat_type","mat_level"]].copy()
    Y = XY[[TARGET_KEY]].astype(float)

if len(X) < 2:
    raise RuntimeError("Not enough rows to train for Toughness. Ensure at least 2 valid samples with non-NaN Toughness and recipe info.")

# ---------- Model pipeline ----------
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])

base = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
reg = Pipeline([("prep", preprocess), ("rf", MultiOutputRegressor(base))])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, test_size=ts, random_state=42)
else:
    X_tr, Y_tr = X, Y
    X_te, Y_te = X.iloc[:0], Y.iloc[:0]

reg.fit(X_tr, Y_tr)

if len(X_te) > 0:
    Y_hat = reg.predict(X_te)
    r2 = r2_score(Y_te.iloc[:, 0], Y_hat[:, 0])
    mae = mean_absolute_error(Y_te.iloc[:, 0], Y_hat[:, 0])
    print(f"[Holdout] Toughness  R2={r2:6.3f}  MAE={mae:8.3f}")
else:
    print("Trained on all rows (small dataset).")

# ---------- Forward design for Toughness ----------
tgt = np.array([TARGET_TOUGHNESS], float)  # single target

# Separate bounds for numeric and handle categoricals
bounds = []
cont_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
cat_cols = [c for c in X.columns if X[c].dtype == "O"]
discrete_enums = {c: sorted(X[c].unique().tolist()) for c in cat_cols}

for c in cont_cols:
    lo, hi = float(X[c].min()), float(X[c].max())
    span = hi - lo if hi > lo else 1.0
    bounds.append((lo - 0.05*span, hi + 0.05*span))

# Build a template row for prediction
def build_row(cont_vec, cat_choice):
    row = {}
    j = 0
    for col in X.columns:
        if col in cat_cols:
            row[col] = cat_choice[col]
        else:
            row[col] = float(cont_vec[j]); j += 1
    return pd.DataFrame([row], columns=X.columns) # Ensure columns order matches X

# Sum-to-constant constraint (if composition percentages)
constraints = []
if MODE == "composition" and COMPOSITION_TOTAL is not None:
    def sum_eq(cont_vec, cont_cols=cont_cols, total=COMPOSITION_TOTAL):
        # This assumes cont_cols are the composition components that sum
        return float(np.sum(cont_vec) - total)
    constraints.append({"type": "eq", "fun": sum_eq})

# penalty for going outside historical range (soft corners)
if cont_cols:
    mins = X[cont_cols].min().values
    maxs = X[cont_cols].max().values
else:
    mins = np.array([])
    maxs = np.array([])


def objective(x, cat_choice): # Modified to accept cat_choice
    row = build_row(x, cat_choice) # Use build_row with cat_choice
    pred = reg.predict(row)[0]   # shape (1,)
    # Weighted MSE + MAPE (weights from train variance, robust to scale)
    var = np.var(Y_tr.values, axis=0) if len(Y_tr) else np.var(Y.values, axis=0)
    w = 1.0 / np.where(var <= 1e-12, 1.0, var)
    eps = 1e-8
    # Use Y.columns directly to slice prediction
    y_pred_targets = pred[[Y.columns.get_loc(k) for k in Y.columns]]
    mse = np.mean(w * (y_pred_targets - tgt)**2)
    mape = np.mean(np.abs(y_pred_targets - tgt) / (np.abs(tgt) + eps))
    base = 0.5*mse + 0.5*mape
    # Soft penalty outside observed domain
    over_low = np.maximum(0.0, mins - x)
    over_high = np.maximum(0.0, x - maxs)
    return base + 1e-3 * np.sum(over_low**2 + over_high**2)

# Start from median composition/knob values
x0_cont = X[cont_cols].median().values if cont_cols else np.array([])

# Enumerate categoricals; minimize continuous for each category combination
from scipy.optimize import minimize
best = {"loss": np.inf, "row": None, "yhat": None}

cat_product = [{}]
if cat_cols:
    import itertools
    cat_values = [discrete_enums[c] for c in cat_cols]
    cat_product = [{c: v for c, v in zip(cat_cols, combo)} for combo in itertools.product(*cat_values)]

for cat_choice in cat_product:
    if not cont_cols:
        # No continuous features — just evaluate categorical combo
        dfrow = build_row(np.array([]), cat_choice)
        # Evaluate loss with objective directly, passing empty array for continuous and the cat_choice
        loss = objective(np.array([]), cat_choice)
        yhat = reg.predict(dfrow)[0]
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
        continue

    def _wrap(x):
        return objective(x, cat_choice) # Pass cat_choice to objective

    result = minimize(
        _wrap, x0_cont,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 1000, "ftol": 1e-9, "disp": False}
    )

    if result.success:
        loss = result.fun
        dfrow = build_row(result.x, cat_choice)
        yhat = reg.predict(dfrow)[0]
        if loss < best["loss"]:
            best.update({"loss": loss, "row": dfrow, "yhat": yhat})
    else:
        print(f"[Warn] Optimization failed for {cat_choice}: {result.message}")


if best["row"] is None:
     raise RuntimeError("Optimization failed for all combinations.")


opt_recipe = best["row"]
opt_pred = pd.DataFrame([best["yhat"]], columns=Y.columns) # Use Y.columns for full prediction columns

print("\n================ RESULTS (Toughness only) ================")
print(f"Mode: {'COMPOSITION → PROPERTIES' if MODE=='composition' else 'LABEL FALLBACK (no composition provided)'}")
print("\nRecommended recipe:")
display_cols = COMPOSITION_COLS if MODE == "composition" else X.columns.tolist() # Define display_cols here
print(opt_recipe[display_cols].to_string(index=False))
print("\nPredicted Toughness_kJ_m3:")
print(opt_pred[[TARGET_KEY]].to_string(index=False)) # Print only the target key

# Save artifacts next to your Drive file
out_dir = Path(DATA_PATH).parent
props_out = out_dir / "_derived_properties_table.csv"
opt_out = out_dir / "_optimal_recipe_Toughness.csv"
props_df.to_csv(props_out, index=False)
pd.concat({"recipe_opt": opt_recipe.reset_index(drop=True),
           "predicted_Toughness": opt_pred[[TARGET_KEY]].reset_index(drop=True)}, axis=1).to_csv(opt_out, index=False) # Save only the target key in the report
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# If you hit "Not enough rows", check coverage:
# print(props_df[["Toughness_kJ_m3"]].notna().sum())

Mounted at /content/drive
[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.


/tmp/ipython-input-505269262.py:98: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))  # kJ/m^3 if stress in kPa and strain is fraction


[Holdout] Toughness  R2=-0.320  MAE=   0.835

================ RESULTS (Toughness only) ================
Mode: LABEL FALLBACK (no composition provided)

Recommended recipe:
mat_type  mat_level
     Cel        5.0

Predicted Toughness_kJ_m3:
 Toughness_kJ_m3
        4.537019

Saved:
  /content/drive/MyDrive/AI Training/_derived_properties_table.csv
  /content/drive/MyDrive/AI Training/_optimal_recipe_Toughness.csv


3) Strain at UTS only

In [17]:
# ===================================================
# One-cell: Strain at UTS only (design recipe/composition)
# ===================================================
!pip -q install numpy pandas scikit-learn scipy

# --- Mount Google Drive for your path ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ---------------- USER CONFIG ----------------
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE): separate file OR a sheet inside DATA_PATH
COMPOSITION_PATH = ""                 # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx" or ".csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet in DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level').
COMPOSITION_COLS: List[str] = [
    # Example: "AAm_wt", "BIS_wt", "APS_wt", "TEMED_wt", "Water_wt"
]

# 4) If composition columns are wt% (or any total constraint), set sum target; else set None.
COMPOSITION_TOTAL = 100.0   # or None if not applicable

# 5) Target STRAIN AT UTS (fraction). Example: 0.18 = 18% strain.
TARGET_STRAIN_UTS = 0.18
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Data pattern: 0, 500, 1000 -> 0.0%, 5.0%, 10.0% -> 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3: continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        # Core properties
        E0_5  = (linear_fit_window(eps, sig, 0.00, 0.05) or (np.nan, np.nan))[0]
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ---------- Load composition (optional) ----------
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ---------- Build X (recipe) and Y (strain at UTS) ----------
TARGET_KEY = "Strain_UTS_frac"  # single target

if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + [TARGET_KEY]].dropna(subset=COMPOSITION_COLS + [TARGET_KEY])
    X = XY[COMPOSITION_COLS].astype(float)
    y = XY[TARGET_KEY].astype(float).values  # 1D vector
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    XY = props_df[["label","mat_type","mat_level", TARGET_KEY]].dropna(subset=[TARGET_KEY])
    X = XY[["mat_type","mat_level"]].copy()
    y = XY[TARGET_KEY].astype(float).values

if len(X) < 2:
    raise RuntimeError("Not enough rows to train for Strain_UTS_frac. Need at least 2 valid samples.")

# ---------- Model pipeline ----------
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])

reg = Pipeline([("prep", preprocess),
                ("rf", RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1))])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=ts, random_state=42)
else:
    X_tr, y_tr = X, y
    X_te, y_te = X.iloc[:0], np.array([])

reg.fit(X_tr, y_tr)

if len(X_te) > 0:
    y_hat = reg.predict(X_te)
    print(f"[Holdout] Strain_UTS_frac  R2={r2_score(y_te, y_hat):6.3f}  MAE={mean_absolute_error(y_te, y_hat):8.4f}")
else:
    print("Trained on all rows (small dataset).")

# ---------- Forward design for Strain_UTS_frac ----------
tgt = float(TARGET_STRAIN_UTS)
mins, maxs = X.min().values, X.max().values

# Helper to evaluate loss for a candidate row
def _loss_from_row(dfrow):
    pred = float(reg.predict(dfrow).ravel()[0])
    # Weighted MSE + MAPE (robust)
    var = float(np.var(y_tr)) if len(y_tr) else float(np.var(y))
    w = 1.0 if var <= 1e-12 else 1.0/var
    eps = 1e-8
    mse = w * (pred - tgt)**2
    mape = abs(pred - tgt) / (abs(tgt) + eps)
    base = 0.5*mse + 0.5*mape
    return base, pred

# ===== Case A: composition mode — continuous optimization over composition columns =====
if MODE == "composition":
    # Bounds with ±5% padding
    bounds = []
    for c in X.columns:
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))

    # Sum-to-constant constraint if set (e.g., wt% totals to 100)
    constraints = []
    if COMPOSITION_TOTAL is not None:
        def sum_eq(x, total=COMPOSITION_TOTAL):
            return float(np.sum(x) - total)
        constraints.append({"type": "eq", "fun": sum_eq})

    def objective(x):
        row = pd.DataFrame([x], columns=X.columns)
        base, _ = _loss_from_row(row)
        # Soft penalty for going outside historical domain
        over_low = np.maximum(0.0, mins - x)
        over_high = np.maximum(0.0, x - maxs)
        return base + 1e-3*np.sum(over_low**2 + over_high**2)

    x0 = X.median().values
    res = minimize(objective, x0, bounds=bounds, constraints=constraints, method="SLSQP",
                   options={"maxiter": 1000, "ftol": 1e-9, "disp": False})
    x_opt = res.x
    opt_row = pd.DataFrame([x_opt], columns=X.columns)
    _, pred_val = _loss_from_row(opt_row)
    mode_label = "COMPOSITION → PROPERTIES"

# ===== Case B: label_fallback — enumerate mat_type (discrete) + optimize mat_level (continuous) =====
else:
    mat_types = sorted(X["mat_type"].unique().tolist())
    level_min, level_max = float(X["mat_level"].min()), float(X["mat_level"].max())
    span = max(1e-9, level_max - level_min)
    bounds = [(level_min - 0.05*span, level_max + 0.05*span)]

    best = {"loss": np.inf, "row": None, "pred": None}

    for mt in mat_types:
        def objective(x):
            row = pd.DataFrame([{"mat_type": mt, "mat_level": float(x[0])}])
            base, _ = _loss_from_row(row)
            # Soft penalty outside observed level range
            over_low = max(0.0, level_min - x[0])
            over_high = max(0.0, x[0] - level_max)
            return base + 1e-3*(over_low**2 + over_high**2)

        x0 = np.array([(level_min + level_max)/2.0])
        res = minimize(objective, x0, bounds=bounds, method="SLSQP",
                       options={"maxiter": 500, "ftol": 1e-9, "disp": False})
        row = pd.DataFrame([{"mat_type": mt, "mat_level": float(res.x[0])}])
        loss, pred = _loss_from_row(row)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": row, "pred": pred})

    opt_row = best["row"]
    pred_val = best["pred"]
    mode_label = "LABEL FALLBACK (no composition provided)"

# ---------- Report & save ----------
print("\n================ RESULTS (Strain at UTS only) ================")
print(f"Mode: {mode_label}")
print("\nRecommended recipe (composition or knobs):")
print(opt_row.to_string(index=False))
print("\nPredicted Strain_UTS_frac:")
print(pd.DataFrame([pred_val], columns=[TARGET_KEY]).to_string(index=False))

# Save artifacts next to your Drive file
out_dir = Path(DATA_PATH).parent
props_out = out_dir / "_derived_properties_table.csv"
opt_out = out_dir / "_optimal_recipe_StrainUTS.csv"
props_df.to_csv(props_out, index=False)
pd.concat({"recipe_opt": opt_row.reset_index(drop=True),
           "predicted_Strain_UTS": pd.DataFrame([pred_val], columns=[TARGET_KEY]).reset_index(drop=True)}, axis=1).to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# If you get "Not enough rows", check coverage:
# print(props_df[[TARGET_KEY]].notna().sum())


Mounted at /content/drive


/tmp/ipython-input-586796648.py:97: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.
[Holdout] Strain_UTS_frac  R2= 0.065  MAE=  0.0071

================ RESULTS (Strain at UTS only) ================
Mode: LABEL FALLBACK (no composition provided)

Recommended recipe (composition or knobs):
mat_type  mat_level
     Cel       10.0

Predicted Strain_UTS_frac:
 Strain_UTS_frac
        0.156046

Saved:
  /content/drive/MyDrive/AI Training/_derived_properties_table.csv
  /content/drive/MyDrive/AI Training/_optimal_recipe_StrainUTS.csv


If you have real composition fields (e.g., AAm_wt, BIS_wt, APS_wt, …), list them in COMPOSITION_COLS and keep COMPOSITION_TOTAL=100.0 if they are wt%. The output then becomes a true chemical recipe satisfying the sum constraint.

Without composition, the code optimizes label-derived knobs (mat_type, mat_level) instead—useful for exploration but not a full chemistry prescription.

Start with a realistic TARGET_STRAIN_UTS within your dataset’s observed range for best results; then explore beyond with caution.

4) Initial modulus (0–5%) only
Here’s a single, copy-paste Colab cell to design a recipe for a target Initial Modulus (0–5%) (E0_5_kPa). It:

reads your file from Google Drive (/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx),

derives properties (including a robust calculation of E0_5_kPa),

trains a model (recipe → E0_5_kPa), and

finds a recipe/composition that hits your target.

It supports either a true composition table (recommended) or a fallback using mat_type + mat_level parsed from labels.

Set your target here: TARGET_E0_5_KPA = 550.0 (example).
If you have an explicit composition table, fill COMPOSITION_PATH or COMPOSITION_SHEET and list numeric columns in COMPOSITION_COLS.

In [18]:
# =====================================================
# One-cell: Initial Modulus (0–5%) only (E0_5_kPa)
# Recipe/composition design to hit a target E0_5
# =====================================================
!pip -q install numpy pandas scikit-learn scipy

# --- Mount Google Drive for your provided path ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ---------------- USER CONFIG ----------------
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE: separate file OR sheet within DATA_PATH)
COMPOSITION_PATH = ""                 # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx" or ".csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet in DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level').
COMPOSITION_COLS: List[str] = [
    # Example: "AAm_wt", "BIS_wt", "APS_wt", "TEMED_wt", "Water_wt"
]

# 4) If composition columns are wt% (or any total constraint), set sum target; else set None.
COMPOSITION_TOTAL = 100.0   # or None if not applicable

# 5) Target Initial Modulus (0–5%), in kPa:
TARGET_E0_5_KPA = 550.0
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Typical pattern: 0, 500, 1000 → 0.0%, 5.0%, 10.0% → 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    """
    Linear fit of σ(ε) in [a,b] using a dense interpolation (20 pts).
    Returns slope (kPa per strain-fraction) and intercept.
    If the full [0, 0.05] window isn't available, we shrink to what's available (down to 1%).
    """
    f, (lo, hi) = interp_curve(eps, sig)
    a_eff = max(lo, a)
    b_eff = min(hi, b)
    if b_eff - a_eff < 0.01:  # need at least ~1% strain span for a stable fit
        return None
    xs = np.linspace(a_eff, b_eff, 20)
    ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    """
    Build a tidy table with per-label properties.
    Crucially, E0_5_kPa is computed by fitting σ(ε) on [0, min(0.05, ε_max)], with a minimum 0.01 window.
    """
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3:
            continue

        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        # --- Initial modulus: fit over [0, min(0.05, hi)]
        fit = linear_fit_window(eps, sig, 0.00, min(0.05, hi))
        E0_5 = fit[0] if fit is not None else np.nan

        # (The rest are optional; computed for context but not used in training)
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ---------- Load composition (optional) ----------
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ---------- Build X (recipe) and y (E0_5_kPa) ----------
TARGET_KEY = "E0_5_kPa"  # single target

if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + [TARGET_KEY]].dropna(subset=COMPOSITION_COLS + [TARGET_KEY])
    X = XY[COMPOSITION_COLS].astype(float)
    y = XY[TARGET_KEY].astype(float).values
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    XY = props_df[["label","mat_type","mat_level", TARGET_KEY]].dropna(subset=[TARGET_KEY])
    X = XY[["mat_type","mat_level"]].copy()
    y = XY[TARGET_KEY].astype(float).values

if len(X) < 2:
    raise RuntimeError("Not enough rows to train for E0_5_kPa. Need at least 2 valid samples.")

# ---------- Model pipeline ----------
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])

reg = Pipeline([
    ("prep", preprocess),
    ("rf", RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1))
])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=ts, random_state=42)
else:
    X_tr, y_tr = X, y
    X_te, y_te = X.iloc[:0], np.array([])

reg.fit(X_tr, y_tr)

if len(X_te) > 0:
    y_hat = reg.predict(X_te)
    print(f"[Holdout] E0_5_kPa  R2={r2_score(y_te, y_hat):6.3f}  MAE={mean_absolute_error(y_te, y_hat):8.3f}")
else:
    print("Trained on all rows (small dataset).")

# ---------- Forward design for E0_5_kPa ----------
tgt = float(TARGET_E0_5_KPA)
mins, maxs = X.min().values, X.max().values

def _loss_from_row(dfrow):
    pred = float(reg.predict(dfrow).ravel()[0])
    # Weighted MSE + MAPE (robust scaling)
    var = float(np.var(y_tr)) if len(y_tr) else float(np.var(y))
    w = 1.0 if var <= 1e-12 else 1.0/var
    eps = 1e-8
    mse = w * (pred - tgt)**2
    mape = abs(pred - tgt) / (abs(tgt) + eps)
    base = 0.5*mse + 0.5*mape
    return base, pred

# ===== Case A: composition mode — continuous optimization over composition columns =====
if MODE == "composition":
    # Bounds with ±5% padding of historical range
    bounds = []
    for c in X.columns:
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))

    # Sum-to-constant constraint if set (e.g., wt% totals to 100)
    constraints = []
    if COMPOSITION_TOTAL is not None:
        def sum_eq(x, total=COMPOSITION_TOTAL):
            return float(np.sum(x) - total)
        constraints.append({"type": "eq", "fun": sum_eq})

    def objective(x):
        row = pd.DataFrame([x], columns=X.columns)
        base, _ = _loss_from_row(row)
        # Soft penalty outside observed domain
        over_low = np.maximum(0.0, mins - x)
        over_high = np.maximum(0.0, x - maxs)
        return base + 1e-3*np.sum(over_low**2 + over_high**2)

    x0 = X.median().values
    res = minimize(objective, x0, bounds=bounds, constraints=constraints, method="SLSQP",
                   options={"maxiter": 1000, "ftol": 1e-9, "disp": False})
    x_opt = res.x
    opt_row = pd.DataFrame([x_opt], columns=X.columns)
    _, pred_val = _loss_from_row(opt_row)
    mode_label = "COMPOSITION → PROPERTIES"

# ===== Case B: label_fallback — enumerate mat_type (discrete) + optimize mat_level (continuous) =====
else:
    mat_types = sorted(X["mat_type"].unique().tolist())
    level_min, level_max = float(X["mat_level"].min()), float(X["mat_level"].max())
    span = max(1e-9, level_max - level_min)
    bounds = [(level_min - 0.05*span, level_max + 0.05*span)]

    best = {"loss": np.inf, "row": None, "pred": None}

    for mt in mat_types:
        def objective(x):
            row = pd.DataFrame([{"mat_type": mt, "mat_level": float(x[0])}])
            base, _ = _loss_from_row(row)
            # Soft penalty outside observed level range
            over_low = max(0.0, level_min - x[0])
            over_high = max(0.0, x[0] - level_max)
            return base + 1e-3*(over_low**2 + over_high**2)

        x0 = np.array([(level_min + level_max)/2.0])
        res = minimize(objective, x0, bounds=bounds, method="SLSQP",
                       options={"maxiter": 500, "ftol": 1e-9, "disp": False})
        row = pd.DataFrame([{"mat_type": mt, "mat_level": float(res.x[0])}])
        loss, pred = _loss_from_row(row)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": row, "pred": pred})

    opt_row = best["row"]
    pred_val = best["pred"]
    mode_label = "LABEL FALLBACK (no composition provided)"

# ---------- Report & save ----------
print("\n================ RESULTS (Initial Modulus 0–5%) ================")
print(f"Mode: {mode_label}")
print("\nRecommended recipe (composition or knobs):")
print(opt_row.to_string(index=False))
print("\nPredicted E0_5_kPa:")
print(pd.DataFrame([pred_val], columns=[TARGET_KEY]).to_string(index=False))

# Save artifacts next to your Drive file
out_dir = Path(DATA_PATH).parent
props_out = out_dir / "_derived_properties_table.csv"
opt_out = out_dir / "_optimal_recipe_E0_5.csv"
props_df.to_csv(props_out, index=False)
pd.concat({"recipe_opt": opt_row.reset_index(drop=True),
           "predicted_E0_5_kPa": pd.DataFrame([pred_val], columns=[TARGET_KEY]).reset_index(drop=True)}, axis=1).to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# If you get "Not enough rows", check coverage:
# print(props_df[[TARGET_KEY]].notna().sum())


Mounted at /content/drive
[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.


/tmp/ipython-input-1708404795.py:105: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


[Holdout] E0_5_kPa  R2=-0.655  MAE=  40.612

================ RESULTS (Initial Modulus 0–5%) ================
Mode: LABEL FALLBACK (no composition provided)

Recommended recipe (composition or knobs):
mat_type  mat_level
     Cel       10.0

Predicted E0_5_kPa:
  E0_5_kPa
384.896667

Saved:
  /content/drive/MyDrive/AI Training/_derived_properties_table.csv
  /content/drive/MyDrive/AI Training/_optimal_recipe_E0_5.csv


Notes

If you have real composition fields (e.g., AAm_wt, BIS_wt, APS_wt, …), list them in COMPOSITION_COLS and keep COMPOSITION_TOTAL=100.0 if they are wt%. Then the output becomes a true chemical composition satisfying the sum constraint.

Without composition, the code optimizes label-derived knobs (mat_type, mat_level) instead—useful for exploration but not a full chemistry prescription.

For best performance, choose a target E0_5_kPa within the observed range first; then explore beyond it.

5) Stress at 10% strain only

Here’s a single, copy-paste Colab cell to design a recipe for a target stress at 10% strain (Stress@10%_kPa). It:

reads your file from Google Drive (/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx),

derives properties (including Stress@10%_kPa),

trains a model (recipe → Stress@10%_kPa), and

finds a recipe/composition that hits your target.

It supports either a true composition table (recommended) or a fallback using mat_type + mat_level parsed from the stress–strain labels.

Set your target here: TARGET_STRESS_10 = 60.0 (example, in kPa).
If you have a composition table, fill COMPOSITION_PATH or COMPOSITION_SHEET and list numeric columns in COMPOSITION_COLS.

In [19]:
# =========================================================
# One-cell: Stress at 10% strain only (Stress@10%_kPa)
# Recipe/composition design to hit a target Stress@10%
# =========================================================
!pip -q install numpy pandas scikit-learn scipy

# --- Mount Google Drive for your provided path ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ---------------- USER CONFIG ----------------
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE: separate file OR sheet within DATA_PATH)
COMPOSITION_PATH = ""                 # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx" or ".csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet in DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level').
COMPOSITION_COLS: List[str] = [
    # Example: "AAm_wt", "BIS_wt", "APS_wt", "TEMED_wt", "Water_wt"
]

# 4) If composition columns are wt% (or any total constraint), set sum target; else set None.
COMPOSITION_TOTAL = 100.0   # or None if not applicable

# 5) Target Stress at 10% strain (kPa):
TARGET_STRESS_10 = 60.0
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Pattern: 0, 500, 1000 → 0.0%, 5.0%, 10.0% → 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3: continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        E0_5  = (linear_fit_window(eps, sig, 0.00, 0.05) or (np.nan, np.nan))[0]
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)

        s5  = stress_at(f, 5, lo, hi)
        s10 = stress_at(f, 10, lo, hi)    # <-- Stress at 10% strain
        s15 = stress_at(f, 15, lo, hi)
        s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5, "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ---------- Load composition (optional) ----------
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ---------- Build X (recipe) and y (Stress@10%_kPa) ----------
TARGET_KEY = "Stress@10%_kPa"  # single target

if TARGET_KEY not in props_df.columns:
    raise RuntimeError("Stress@10%_kPa not computed — your curves might not reach 10% strain.")

if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + [TARGET_KEY]].dropna(subset=COMPOSITION_COLS + [TARGET_KEY])
    X = XY[COMPOSITION_COLS].astype(float)
    y = XY[TARGET_KEY].astype(float).values
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    XY = props_df[["label","mat_type","mat_level", TARGET_KEY]].dropna(subset=[TARGET_KEY])
    X = XY[["mat_type","mat_level"]].copy()
    y = XY[TARGET_KEY].astype(float).values

if len(X) < 2:
    raise RuntimeError("Not enough rows to train for Stress@10%_kPa. Need at least 2 valid samples with this property.")

# ---------- Model pipeline ----------
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])

reg = Pipeline([
    ("prep", preprocess),
    ("rf", RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1))
])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=ts, random_state=42)
else:
    X_tr, y_tr = X, y
    X_te, y_te = X.iloc[:0], np.array([])

reg.fit(X_tr, y_tr)

if len(X_te) > 0:
    y_hat = reg.predict(X_te)
    print(f"[Holdout] Stress@10%_kPa  R2={r2_score(y_te, y_hat):6.3f}  MAE={mean_absolute_error(y_te, y_hat):8.3f}")
else:
    print("Trained on all rows (small dataset).")

# ---------- Forward design for Stress@10%_kPa ----------
tgt = float(TARGET_STRESS_10)
mins, maxs = X.min().values, X.max().values

def _loss_from_row(dfrow):
    pred = float(reg.predict(dfrow).ravel()[0])
    # Weighted MSE + MAPE (robust scaling)
    var = float(np.var(y_tr)) if len(y_tr) else float(np.var(y))
    w = 1.0 if var <= 1e-12 else 1.0/var
    eps = 1e-8
    mse = w * (pred - tgt)**2
    mape = abs(pred - tgt) / (abs(tgt) + eps)
    base = 0.5*mse + 0.5*mape
    return base, pred

# ===== Case A: composition mode — continuous optimization over composition columns =====
if MODE == "composition":
    # Bounds with ±5% padding of historical range
    bounds = []
    for c in X.columns:
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))

    # Sum-to-constant constraint if set (e.g., wt% totals to 100)
    constraints = []
    if COMPOSITION_TOTAL is not None:
        def sum_eq(x, total=COMPOSITION_TOTAL):
            return float(np.sum(x) - total)
        constraints.append({"type": "eq", "fun": sum_eq})

    def objective(x):
        row = pd.DataFrame([x], columns=X.columns)
        base, _ = _loss_from_row(row)
        # Soft penalty outside observed domain
        over_low = np.maximum(0.0, mins - x)
        over_high = np.maximum(0.0, x - maxs)
        return base + 1e-3*np.sum(over_low**2 + over_high**2)

    x0 = X.median().values
    res = minimize(objective, x0, bounds=bounds, constraints=constraints, method="SLSQP",
                   options={"maxiter": 1000, "ftol": 1e-9, "disp": False})
    x_opt = res.x
    opt_row = pd.DataFrame([x_opt], columns=X.columns)
    _, pred_val = _loss_from_row(opt_row)
    mode_label = "COMPOSITION → PROPERTIES"

# ===== Case B: label_fallback — enumerate mat_type (discrete) + optimize mat_level (continuous) =====
else:
    mat_types = sorted(X["mat_type"].unique().tolist())
    level_min, level_max = float(X["mat_level"].min()), float(X["mat_level"].max())
    span = max(1e-9, level_max - level_min)
    bounds = [(level_min - 0.05*span, level_max + 0.05*span)]

    best = {"loss": np.inf, "row": None, "pred": None}

    for mt in mat_types:
        def objective(x):
            row = pd.DataFrame([{"mat_type": mt, "mat_level": float(x[0])}])
            base, _ = _loss_from_row(row)
            # Soft penalty outside observed level range
            over_low = max(0.0, level_min - x[0])
            over_high = max(0.0, x[0] - level_max)
            return base + 1e-3*(over_low**2 + over_high**2)

        x0 = np.array([(level_min + level_max)/2.0])
        res = minimize(objective, x0, bounds=bounds, method="SLSQP",
                       options={"maxiter": 500, "ftol": 1e-9, "disp": False})
        row = pd.DataFrame([{"mat_type": mt, "mat_level": float(res.x[0])}])
        loss, pred = _loss_from_row(row)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": row, "pred": pred})

    opt_row = best["row"]
    pred_val = best["pred"]
    mode_label = "LABEL FALLBACK (no composition provided)"

# ---------- Report & save ----------
print("\n================ RESULTS (Stress at 10% strain) ================")
print(f"Mode: {mode_label}")
print("\nRecommended recipe (composition or knobs):")
print(opt_row.to_string(index=False))
print("\nPredicted Stress@10%_kPa:")
print(pd.DataFrame([pred_val], columns=[TARGET_KEY]).to_string(index=False))

# Save artifacts next to your Drive file
out_dir = Path(DATA_PATH).parent
props_out = out_dir / "_derived_properties_table.csv"
opt_out = out_dir / "_optimal_recipe_Stress10.csv"
props_df.to_csv(props_out, index=False)
pd.concat({"recipe_opt": opt_row.reset_index(drop=True),
           "predicted_Stress10": pd.DataFrame([pred_val], columns=[TARGET_KEY]).reset_index(drop=True)}, axis=1).to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# If you get "Not enough rows", your curves might not reach 10% strain.
# You can check coverage with:
# print(props_df[[TARGET_KEY]].notna().sum())


Mounted at /content/drive
[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.


/tmp/ipython-input-1753915685.py:98: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


[Holdout] Stress@10%_kPa  R2=-2.135  MAE=   4.558

================ RESULTS (Stress at 10% strain) ================
Mode: LABEL FALLBACK (no composition provided)

Recommended recipe (composition or knobs):
mat_type  mat_level
     Cel       10.0

Predicted Stress@10%_kPa:
 Stress@10%_kPa
        35.2375

Saved:
  /content/drive/MyDrive/AI Training/_derived_properties_table.csv
  /content/drive/MyDrive/AI Training/_optimal_recipe_Stress10.csv


Notes

For true chemical output, provide your composition table and list its numeric columns in COMPOSITION_COLS. Keep COMPOSITION_TOTAL=100.0 if those are wt%.

Without composition, the optimizer proposes label-derived knobs (mat_type, mat_level) instead—useful for exploration but not full chemistry.

If you see “Not enough rows,” it likely means many curves don’t reach 10% strain; try targeting Stress@5%_kPa first to confirm the pipeline, or expand your data that covers 10%.

6) Yield stress (0.2% offset) only
Here’s a single, copy-paste Colab cell to design a recipe for a target Yield stress (0.2% offset) using your Drive file:

reads /content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx

derives Yield_stress_kPa (and Yield_strain_frac) via 0.2% offset using an initial 0–5% modulus fit

trains a model (recipe → Yield_stress_kPa)

optimizes a composition (if you provide a composition table) or label-fallback knobs (mat_type, mat_level) to hit your target

Set your target here: TARGET_YIELD_STRESS = 25.0 (kPa).
If you have a composition table, fill COMPOSITION_PATH or COMPOSITION_SHEET and list numeric columns in COMPOSITION_COLS (e.g., wt%). Keep COMPOSITION_TOTAL=100.0 if those are percentages.

In [22]:
# ============================================================
# One-cell: Yield stress (0.2% offset) only — design recipe
# ============================================================
!pip -q install numpy pandas scikit-learn scipy

# --- Mount Google Drive for your provided path ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ---------------- USER CONFIG ----------------
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE: separate file OR sheet within DATA_PATH)
COMPOSITION_PATH = ""                 # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx" or ".csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet in DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level').
COMPOSITION_COLS: List[str] = [
    # Example: "AAm_wt", "BIS_wt", "APS_wt", "TEMED_wt", "Water_wt"
]

# 4) If composition columns are wt% (or any total constraint), set sum target; else set None.
COMPOSITION_TOTAL = 100.0   # or None if not applicable

# 5) Target Yield stress (kPa):
TARGET_YIELD_STRESS = 25.0
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Pattern: 0, 500, 1000 → 0.0%, 5.0%, 10.0% → 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    """
    Linear fit of σ(ε) in [a,b] using dense interpolation.
    If [0,0.05] isn't fully available, shrink to what's available, with a minimum 0.01 span.
    """
    f, (lo, hi) = interp_curve(eps, sig)
    a_eff, b_eff = max(lo, a), min(hi, b)
    if b_eff - a_eff < 0.01:
        return None
    xs = np.linspace(a_eff, b_eff, 20)
    ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    """
    0.2% offset yield: intersection of σ(ε) with line E_init*(ε - offset).
    Returns (yield_strain, yield_stress) or (None, None) if no intersection.
    """
    if E_init is None or not np.isfinite(E_init):
        return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 600)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g)
    idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0:
        return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    """
    Build a tidy table with per-label properties.
    Crucially, E0_5_kPa is computed over [0, min(0.05, ε_max)], with a minimum 0.01 span.
    Yield is computed via 0.2% offset using that E0_5_kPa.
    """
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3:
            continue

        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        # Initial modulus over [0, min(0.05, hi)] with min span 0.01
        fit = linear_fit_window(eps, sig, 0.00, min(0.05, hi))
        E0_5 = fit[0] if fit is not None else np.nan

        # Yield via 0.2% offset
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)

        # Extra properties (optional context)
        E5_10 = (lambda f=f: (float((f(0.10)-f(0.05))/0.05) if hi>=0.10 else np.nan))()
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)
        uts = float(sig.max()); strain_uts = float(eps[int(sig.argmax())])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        sec_0_15 = (lambda f=f: (float((f(0.15)-f(0.00))/0.15) if hi>=0.15 else np.nan))()


        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            # Optional context fields (not used for training unless you want to)
            "E5_10_kPa": E5_10, "TanE10_kPa": TanE10,
            "Resilience_kJ_m3":  resilience,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "Secant_0_15_kPa": sec_0_15
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ---------- Load composition (optional) ----------
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ---------- Build X (recipe) and y (Yield_stress_kPa) ----------
TARGET_KEY = "Yield_stress_kPa"  # single target

if TARGET_KEY not in props_df.columns:
    raise RuntimeError("Yield_stress_kPa not computed — curves may not permit 0.2% offset yield (or initial modulus window too small).")

if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + [TARGET_KEY]].dropna(subset=COMPOSITION_COLS + [TARGET_KEY])
    X = XY[COMPOSITION_COLS].astype(float)
    y = XY[TARGET_KEY].astype(float).values
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    XY = props_df[["label","mat_type","mat_level", TARGET_KEY]].dropna(subset=[TARGET_KEY])
    X = XY[["mat_type","mat_level"]].copy()
    y = XY[TARGET_KEY].astype(float).values

if len(X) < 2:
    raise RuntimeError("Not enough rows to train for Yield_stress_kPa. Need at least 2 valid samples with this property.")

# ---------- Model pipeline ----------
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])

reg = Pipeline([
    ("prep", preprocess),
    ("rf", RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1))
])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=ts, random_state=42)
else:
    X_tr, y_tr = X, y
    X_te, y_te = X.iloc[:0], np.array([])

reg.fit(X_tr, y_tr)

if len(X_te) > 0:
    y_hat = reg.predict(X_te)
    print(f"[Holdout] Yield_stress_kPa  R2={r2_score(y_te, y_hat):6.3f}  MAE={mean_absolute_error(y_te, y_hat):8.3f}")
else:
    print("Trained on all rows (small dataset).")

# ---------- Forward design for Yield_stress_kPa ----------
tgt = float(TARGET_YIELD_STRESS)
# Ensure mins and maxs are correctly sliced for continuous columns
if cont_cols: # Check if cont_cols is not empty
  mins, maxs = X[cont_cols].min().values, X[cont_cols].max().values
else:
  mins, maxs = np.array([]), np.array([])


def _loss_from_row(dfrow):
    pred = float(reg.predict(dfrow).ravel()[0])
    # Weighted MSE + MAPE (robust scaling)
    var = float(np.var(y_tr)) if len(y_tr) else float(np.var(y))
    w = 1.0 if var <= 1e-12 else 1.0/var
    eps = 1e-8
    mse = w * (pred - tgt)**2
    mape = abs(pred - tgt) / (abs(tgt) + eps)
    base = 0.5*mse + 0.5*mape
    return base, pred

# ===== Case A: composition mode — continuous optimization over composition columns =====
if MODE == "composition":
    # Bounds with ±5% padding of historical range
    bounds = []
    for c in X.columns:
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))

    # Sum-to-constant constraint if set (e.g., wt% totals to 100)
    constraints = []
    if COMPOSITION_TOTAL is not None:
        def sum_eq(x, total=COMPOSITION_TOTAL):
            return float(np.sum(x) - total)
        constraints.append({"type": "eq", "fun": sum_eq})

    def objective(x):
        row = pd.DataFrame([x], columns=X.columns)
        base, _ = _loss_from_row(row)
        # Soft penalty outside observed domain
        over_low = np.maximum(0.0, mins - x)
        over_high = np.maximum(0.0, x - maxs)
        return base + 1e-3*np.sum(over_low**2 + over_high**2)

    x0 = X.median().values
    res = minimize(objective, x0, bounds=bounds, constraints=constraints, method="SLSQP",
                   options={"maxiter": 1000, "ftol": 1e-9, "disp": False})
    x_opt = res.x
    opt_row = pd.DataFrame([x_opt], columns=X.columns)
    _, pred_val = _loss_from_row(opt_row)
    mode_label = "COMPOSITION → PROPERTIES"

# ===== Case B: label_fallback — enumerate mat_type (discrete) + optimize mat_level (continuous) =====
else:
    mat_types = sorted(X["mat_type"].unique().tolist())
    level_min, level_max = float(X["mat_level"].min()), float(X["mat_level"].max())
    span = max(1e-9, level_max - level_min)
    bounds = [(level_min - 0.05*span, level_max + 0.05*span)]

    best = {"loss": np.inf, "row": None, "pred": None}

    for mt in mat_types:
        def objective(x):
            row = pd.DataFrame([{"mat_type": mt, "mat_level": float(x[0])}])
            base, _ = _loss_from_row(row)
            # Soft penalty outside observed level range
            over_low = max(0.0, level_min - x[0])
            over_high = max(0.0, x[0] - level_max)
            return base + 1e-3*(over_low**2 + over_high**2)

        x0 = np.array([(level_min + level_max)/2.0])
        res = minimize(objective, x0, bounds=bounds, method="SLSQP",
                       options={"maxiter": 500, "ftol": 1e-9, "disp": False})
        row = pd.DataFrame([{"mat_type": mt, "mat_level": float(res.x[0])}])
        loss, pred = _loss_from_row(row)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": row, "pred": pred})

    opt_row = best["row"]
    pred_val = best["pred"]
    mode_label = "LABEL FALLBACK (no composition provided)"

# ---------- Report & save ----------
print("\n================ RESULTS (Yield stress 0.2% offset) ================")
print(f"Mode: {mode_label}")
print("\nRecommended recipe (composition or knobs):")
print(opt_row.to_string(index=False))
print("\nPredicted Yield_stress_kPa:")
print(pd.DataFrame([pred_val], columns=[TARGET_KEY]).to_string(index=False))

# Save artifacts next to your Drive file
out_dir = Path(DATA_PATH).parent
props_out = out_dir / "_derived_properties_table.csv"
opt_out = out_dir / "_optimal_recipe_YieldStress.csv"
props_df.to_csv(props_out, index=False)
pd.concat({"recipe_opt": opt_row.reset_index(drop=True),
           "predicted_Yield_stress": pd.DataFrame([pred_val], columns=[TARGET_KEY]).reset_index(drop=True)}, axis=1).to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# If you get "Not enough rows", it means few curves have a resolvable 0.2% yield.
# Try: print(props_df[['E0_5_kPa','Yield_stress_kPa','Yield_strain_frac']].notna().sum())

Mounted at /content/drive
[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.


/tmp/ipython-input-3797196544.py:105: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


[Holdout] Yield_stress_kPa  R2=-0.017  MAE=   4.404

================ RESULTS (Yield stress 0.2% offset) ================
Mode: LABEL FALLBACK (no composition provided)

Recommended recipe (composition or knobs):
mat_type  mat_level
     Cel       10.0

Predicted Yield_stress_kPa:
 Yield_stress_kPa
        25.124281

Saved:
  /content/drive/MyDrive/AI Training/_derived_properties_table.csv
  /content/drive/MyDrive/AI Training/_optimal_recipe_YieldStress.csv


If you want the output to be a true chemical composition, provide COMPOSITION_COLS and keep COMPOSITION_TOTAL=100.0 for wt%.

If many curves don’t resolve a 0.2% offset yield, consider tightening your data near 0–2% strain (denser sampling), or temporarily target a proxy like Stress@5%_kPa to validate the pipeline.

7) Fracture strain only
Here’s a single, copy-paste Colab cell to design a recipe for a target Fracture strain (Fracture_strain_frac). It:

reads your Drive file,

derives properties (including Fracture_strain_frac),

trains a model (recipe → Fracture_strain_frac),

optimizes either a true composition (if you provide a composition table) or the label-fallback knobs (mat_type, mat_level) to hit your target.

Set your target here: TARGET_FRACTURE_STRAIN = 0.20 (example = 20% strain).
If you have a composition table, fill COMPOSITION_PATH or COMPOSITION_SHEET and list numeric columns in COMPOSITION_COLS. Keep COMPOSITION_TOTAL=100.0 if those are wt%.

In [23]:
# =======================================================
# One-cell: Fracture strain only (design recipe/composition)
# Target property: Fracture_strain_frac  (unitless fraction)
# =======================================================
!pip -q install numpy pandas scikit-learn scipy

# --- Mount Google Drive for your provided path ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional, List
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.optimize import minimize

# ---------------- USER CONFIG ----------------
# 1) Stress–strain file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# 2) Composition source (choose ONE: separate file OR sheet within DATA_PATH)
COMPOSITION_PATH = ""                 # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx" or ".csv"
COMPOSITION_SHEET = ""                # e.g., "composition" if stored as a sheet in DATA_PATH

# 3) Composition columns (true chemistry knobs). Leave empty to fallback to ('mat_type','mat_level').
COMPOSITION_COLS: List[str] = [
    # Example: "AAm_wt", "BIS_wt", "APS_wt", "TEMED_wt", "Water_wt"
]

# 4) If composition columns are wt% (or any total constraint), set sum target; else set None.
COMPOSITION_TOTAL = 100.0   # or None if not applicable

# 5) Target Fracture strain (fraction). Example: 0.20 = 20% strain.
TARGET_FRACTURE_STRAIN = 0.20
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # Pattern: 0, 500, 1000 → 0.0%, 5.0%, 10.0% → 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float)/100.0)/100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    if b <= a: return None
    f, (lo, hi) = interp_curve(eps, sig)
    a, b = max(lo, a), min(hi, b)
    if b <= a: return None
    xs = np.linspace(a, b, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def secant_modulus(f, a: float, b: float):
    if b <= a: return np.nan
    return float((f(b) - f(a)) / (b - a))

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 400)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

def parse_label_recipe(label: str):
    import re
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3: continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        # Initial modulus (for context; not used for training)
        E0_5  = (linear_fit_window(eps, sig, 0.00, min(0.05, hi)) or (np.nan, np.nan))[0]
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)

        # Key quantities
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max())           # <-- Fracture strain (max recorded strain)
        frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)

        # Convenience probes
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)
        E5_10 = secant_modulus(f, 0.05, 0.10) if hi >= 0.10 else np.nan
        TanE10 = (linear_fit_window(eps, sig, 0.08, 0.12) or (np.nan, np.nan))[0] if hi >= 0.12 else np.nan
        sec_0_15 = secant_modulus(f, 0.00, 0.15) if hi >= 0.15 else np.nan
        resilience = integrate_toughness(eps, sig, up_to=eps_y) if eps_y is not None else (integrate_toughness(eps, sig, up_to=0.02) if hi >= 0.02 else np.nan)

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain,   # <-- target column
            "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20,
            "E5_10_kPa": E5_10, "TanE10_kPa": TanE10, "Secant_0_15_kPa": sec_0_15,
            "Resilience_kJ_m3": resilience
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

# ---------- Load composition (optional) ----------
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        if COMPOSITION_PATH.lower().endswith(".csv"):
            comp_df = pd.read_csv(COMPOSITION_PATH)
        else:
            comp_df = pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

# ---------- Build X (recipe) and y (Fracture_strain_frac) ----------
TARGET_KEY = "Fracture_strain_frac"  # single target

if TARGET_KEY not in props_df.columns:
    raise RuntimeError("Fracture_strain_frac not computed.")

if comp_df is not None and len(COMPOSITION_COLS) > 0 and all(c in comp_df.columns for c in COMPOSITION_COLS) and "label" in comp_df.columns:
    MODE = "composition"
    df_model = props_df.merge(comp_df[["label"] + COMPOSITION_COLS], on="label", how="inner")
    XY = df_model[["label"] + COMPOSITION_COLS + [TARGET_KEY]].dropna(subset=COMPOSITION_COLS + [TARGET_KEY])
    X = XY[COMPOSITION_COLS].astype(float)
    y = XY[TARGET_KEY].astype(float).values
else:
    MODE = "label_fallback"
    print("[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.")
    XY = props_df[["label","mat_type","mat_level", TARGET_KEY]].dropna(subset=[TARGET_KEY])
    X = XY[["mat_type","mat_level"]].copy()
    y = XY[TARGET_KEY].astype(float).values

if len(X) < 2:
    raise RuntimeError("Not enough rows to train for Fracture_strain_frac. Need at least 2 valid samples.")

# ---------- Model pipeline ----------
if MODE == "composition":
    preprocess = ColumnTransformer([("num", StandardScaler(), X.columns.tolist())])
else:
    preprocess = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["mat_type"]),
        ("num", StandardScaler(), ["mat_level"]),
    ])

reg = Pipeline([
    ("prep", preprocess),
    ("rf", RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1))
])

ts = 0.33 if len(X) >= 6 else 0.0
if ts > 0:
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=ts, random_state=42)
else:
    X_tr, y_tr = X, y
    X_te, y_te = X.iloc[:0], np.array([])

reg.fit(X_tr, y_tr)

if len(X_te) > 0:
    y_hat = reg.predict(X_te)
    print(f"[Holdout] Fracture_strain_frac  R2={r2_score(y_te, y_hat):6.3f}  MAE={mean_absolute_error(y_te, y_hat):8.4f}")
else:
    print("Trained on all rows (small dataset).")

# ---------- Forward design for Fracture_strain_frac ----------
tgt = float(TARGET_FRACTURE_STRAIN)
mins, maxs = X.min().values, X.max().values

def _loss_from_row(dfrow):
    pred = float(reg.predict(dfrow).ravel()[0])
    # Weighted MSE + MAPE (robust scaling)
    var = float(np.var(y_tr)) if len(y_tr) else float(np.var(y))
    w = 1.0 if var <= 1e-12 else 1.0/var
    eps = 1e-8
    mse = w * (pred - tgt)**2
    mape = abs(pred - tgt) / (abs(tgt) + eps)
    base = 0.5*mse + 0.5*mape
    return base, pred

# ===== Case A: composition mode — continuous optimization over composition columns =====
if MODE == "composition":
    # Bounds with ±5% padding of historical range
    bounds = []
    for c in X.columns:
        lo, hi = float(X[c].min()), float(X[c].max())
        span = hi - lo if hi > lo else 1.0
        bounds.append((lo - 0.05*span, hi + 0.05*span))

    # Sum-to-constant constraint if set (e.g., wt% totals to 100)
    constraints = []
    if COMPOSITION_TOTAL is not None:
        def sum_eq(x, total=COMPOSITION_TOTAL):
            return float(np.sum(x) - total)
        constraints.append({"type": "eq", "fun": sum_eq})

    def objective(x):
        row = pd.DataFrame([x], columns=X.columns)
        base, _ = _loss_from_row(row)
        # Soft penalty outside observed domain
        over_low = np.maximum(0.0, mins - x)
        over_high = np.maximum(0.0, x - maxs)
        return base + 1e-3*np.sum(over_low**2 + over_high**2)

    x0 = X.median().values
    res = minimize(objective, x0, bounds=bounds, constraints=constraints, method="SLSQP",
                   options={"maxiter": 1000, "ftol": 1e-9, "disp": False})
    x_opt = res.x
    opt_row = pd.DataFrame([x_opt], columns=X.columns)
    _, pred_val = _loss_from_row(opt_row)
    mode_label = "COMPOSITION → PROPERTIES"

# ===== Case B: label_fallback — enumerate mat_type (discrete) + optimize mat_level (continuous) =====
else:
    mat_types = sorted(X["mat_type"].unique().tolist())
    level_min, level_max = float(X["mat_level"].min()), float(X["mat_level"].max())
    span = max(1e-9, level_max - level_min)
    bounds = [(level_min - 0.05*span, level_max + 0.05*span)]

    best = {"loss": np.inf, "row": None, "pred": None}

    for mt in mat_types:
        def objective(x):
            row = pd.DataFrame([{"mat_type": mt, "mat_level": float(x[0])}])
            base, _ = _loss_from_row(row)
            # Soft penalty outside observed level range
            over_low = max(0.0, level_min - x[0])
            over_high = max(0.0, x[0] - level_max)
            return base + 1e-3*(over_low**2 + over_high**2)

        x0 = np.array([(level_min + level_max)/2.0])
        res = minimize(objective, x0, bounds=bounds, method="SLSQP",
                       options={"maxiter": 500, "ftol": 1e-9, "disp": False})
        row = pd.DataFrame([{"mat_type": mt, "mat_level": float(res.x[0])}])
        loss, pred = _loss_from_row(row)
        if loss < best["loss"]:
            best.update({"loss": loss, "row": row, "pred": pred})

    opt_row = best["row"]
    pred_val = best["pred"]
    mode_label = "LABEL FALLBACK (no composition provided)"

# ---------- Report & save ----------
print("\n================ RESULTS (Fracture strain) ================")
print(f"Mode: {mode_label}")
print("\nRecommended recipe (composition or knobs):")
print(opt_row.to_string(index=False))
print("\nPredicted Fracture_strain_frac:")
print(pd.DataFrame([pred_val], columns=[TARGET_KEY]).to_string(index=False))

# Save artifacts next to your Drive file
out_dir = Path(DATA_PATH).parent
props_out = out_dir / "_derived_properties_table.csv"
opt_out = out_dir / "_optimal_recipe_FractureStrain.csv"
props_df.to_csv(props_out, index=False)
pd.concat({"recipe_opt": opt_row.reset_index(drop=True),
           "predicted_Fracture_strain": pd.DataFrame([pred_val], columns=[TARGET_KEY]).reset_index(drop=True)}, axis=1).to_csv(opt_out, index=False)
print(f"\nSaved:\n  {props_out}\n  {opt_out}")

# If you get "Not enough rows", check coverage:
# print(props_df[[TARGET_KEY]].notna().sum())


Mounted at /content/drive
[Warn] No composition table recognized. Falling back to ('mat_type','mat_level') from labels.


/tmp/ipython-input-1000350996.py:98: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))


[Holdout] Fracture_strain_frac  R2= 0.065  MAE=  0.0071

================ RESULTS (Fracture strain) ================
Mode: LABEL FALLBACK (no composition provided)

Recommended recipe (composition or knobs):
mat_type  mat_level
     Cel       10.0

Predicted Fracture_strain_frac:
 Fracture_strain_frac
             0.156046

Saved:
  /content/drive/MyDrive/AI Training/_derived_properties_table.csv
  /content/drive/MyDrive/AI Training/_optimal_recipe_FractureStrain.csv


চলুন Figure 5(E)-র মতো 3D ডায়াগ্রাম বানাই—আপনার experimental data থেকে (stress–strain → derived properties) আর (থাকলে) composition টেবিল থেকে AM–MBA–APS 3D scatter। নিচের একটাই Colab সেল কপি–পেস্ট করলেই হবে। এতে—

আপনার Excel ফাইল থেকে properties বের করে 3D property–property–property scatter দেয়

যদি আলাদা composition টেবিল থাকে (AM, MBA, APS + gelation/class), তাহলে Figure 5(E)-স্টাইল 3D composition scatter দেয় (জেল হয়েছে/হয়নি—রঙে আলাদা)

rbae109

যে পেপারটির Figure 5(E) রেফারেন্স করা হয়েছে, সেখানে AM–MBA–APS তিন ফিচারের 3D স্ক্যাটার দেখানো হয়েছে জেলেশন ক্লাস দিয়ে কালার-কোড করে। এখানেও সেই ধারণাটা ফলো করছি।

paper link: https://drive.google.com/file/d/1ZRInPfNQ5L0DPeB9vxKJLGCNR8gCI3pY/view

In [24]:
# ================================================
# Colab one-cell: 3D diagrams like Fig. 5(E)
# - 3D Property scatter from stress–strain data
# - (Optional) 3D Composition scatter: AM–MBA–APS with gelation class
# ================================================
!pip -q install numpy pandas plotly scikit-learn scipy

# ---- Mount Google Drive (needed for your given path) ----
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
from typing import Dict, Tuple, Optional, List
from sklearn.preprocessing import MinMaxScaler

# ---------------- USER CONFIG ----------------
# Your stress–strain Excel file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# (Optional) Composition source:
#   Option A: a separate file (.xlsx/.csv)
COMPOSITION_PATH = ""   # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx"
#   Option B: a sheet inside DATA_PATH
COMPOSITION_SHEET = ""  # e.g., "composition"

# Columns we’ll look for to replicate Fig.5(E)-style AM–MBA–APS plot (case-insensitive match done automatically)
LIKELY_AM_COLS  = ["am", "aam", "acrylamide", "am_wt", "aam_wt"]
LIKELY_MBA_COLS = ["mba", "bis", "n,n-methylenebisacrylamide", "bis_wt", "mba_wt"]
LIKELY_APS_COLS = ["aps", "ammonium persulfate", "aps_wt"]

# Optional gelation/class column names we’ll search for
LIKELY_CLASS_COLS = ["gel_ok", "gelation", "class", "label_y", "is_gel", "formed", "status"]

# Default 3D property axes (you can change these names to any derived property columns below)
PROPERTY_X = "UTS_kPa"
PROPERTY_Y = "Fracture_strain_frac"   # unitless fraction
PROPERTY_Z = "Toughness_kJ_m3"
# --------------------------------------------

# ======== Stress–strain utilities ========
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # e.g., 0, 500, 1000 -> 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float) / 100.0) / 100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    f, (lo, hi) = interp_curve(eps, sig)
    a_eff, b_eff = max(lo, a), min(hi, b)
    if b_eff - a_eff < 0.01:  # need at least ~1% strain span
        return None
    xs = np.linspace(a_eff, b_eff, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 600)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))  # stress(kPa) × strain(fraction) -> kJ/m^3 (since kPa = kJ/m^3)

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

import re
def parse_label_recipe(label: str):
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3:
            continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        # Initial modulus over [0, min(0.05, hi)] with min 0.01 span
        fit = linear_fit_window(eps, sig, 0.00, min(0.05, hi))
        E0_5 = fit[0] if fit is not None else np.nan

        # Yield via 0.2% offset
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)

        # Key properties
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20
        })
    return pd.DataFrame(rows)

# ---------- Derive properties from your stress–strain file ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

display_cols = [
    "label","mat_type","mat_level","UTS_kPa","Strain_UTS_frac","Fracture_strain_frac",
    "Fracture_stress_kPa","Toughness_kJ_m3","E0_5_kPa","Yield_stress_kPa","Yield_strain_frac",
    "Stress@5%_kPa","Stress@10%_kPa","Stress@15%_kPa","Stress@20%_kPa"
]
print("Derived property table (head):")
display(props_df[display_cols].head())

# ---------- 3D Property scatter (choose axes above) ----------
for col in [PROPERTY_X, PROPERTY_Y, PROPERTY_Z]:
    if col not in props_df.columns:
        raise RuntimeError(f"Property axis '{col}' not found in derived table. Available columns: {list(props_df.columns)}")

fig_prop = px.scatter_3d(
    props_df, x=PROPERTY_X, y=PROPERTY_Y, z=PROPERTY_Z,
    color="mat_type",
    symbol="mat_type",
    size_max=10,
    hover_data=["label","mat_level","E0_5_kPa","Yield_stress_kPa","Stress@10%_kPa","Toughness_kJ_m3"]
)
fig_prop.update_traces(marker=dict(size=6, opacity=0.8))
fig_prop.update_layout(
    title=f"3D Property Space: {PROPERTY_X} vs {PROPERTY_Y} vs {PROPERTY_Z}",
    legend_title="mat_type"
)
fig_prop.show()

# ---------- Try to build Fig. 5(E)-style 3D Composition scatter ----------
def load_composition_table() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        comp_df = pd.read_csv(COMPOSITION_PATH) if COMPOSITION_PATH.lower().endswith(".csv") else pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

def pick_first_match(cols, candidates):
    cl = {c.lower(): c for c in cols}
    for name in candidates:
        if name in cl:
            return cl[name]
    # try startswith match
    for name in candidates:
        for lc, orig in cl.items():
            if lc.startswith(name):
                return orig
    return None

comp_df = load_composition_table()
if comp_df is not None:
    # normalize column names to lower for matching
    lower_map = {c.lower(): c for c in comp_df.columns}
    am_col  = pick_first_match(lower_map.keys(), LIKELY_AM_COLS)
    mba_col = pick_first_match(lower_map.keys(), LIKELY_MBA_COLS)
    aps_col = pick_first_match(lower_map.keys(), LIKELY_APS_COLS)
    cls_col = pick_first_match(lower_map.keys(), LIKELY_CLASS_COLS)

    # Merge with properties on 'label' if that exists in both
    if "label" in comp_df.columns and "label" in props_df.columns:
        merged = props_df.merge(comp_df, on="label", how="inner")
    else:
        merged = comp_df.copy()

    if am_col and mba_col and aps_col:
        plot_df = merged.copy()
        # Make a friendly class for coloring if available
        if cls_col and cls_col in plot_df.columns:
            # Coerce to 0/1 if needed
            v = plot_df[cls_col]
            if v.dtype == bool:
                cls = v.map({True:"Gel formed", False:"Cutoff"})
            else:
                # numeric or string—treat nonzero/yes-like as formed
                cls = v.astype(str).str.lower().map(
                    lambda s: "Gel formed" if s in ["1","true","yes","y","ok","formed","gel","good"] else ("Cutoff" if s in ["0","false","no","n","cutoff","bad","fail"] else "Gel formed")
                )
            plot_df["_gel_class_"] = cls
            color_col = "_gel_class_"
        else:
            color_col = None  # no class info -> single color

        fig_comp = px.scatter_3d(
            plot_df, x=am_col, y=mba_col, z=aps_col,
            color=color_col if color_col else None,
            symbol=color_col if color_col else None,
            hover_data=["label"] if "label" in plot_df.columns else None
        )
        fig_comp.update_traces(marker=dict(size=5, opacity=0.85))
        ttl = f"3D Composition Space ({am_col} vs {mba_col} vs {aps_col})"
        if color_col:
            ttl += " — gelation class"
        fig_comp.update_layout(title=ttl)
        fig_comp.show()
    else:
        print("⚠️ Composition table loaded, but AM/MBA/APS columns not found. "
              "Set COMPOSITION_PATH/COMPOSITION_SHEET correctly and ensure columns like AM, MBA, APS exist.")
else:
    print("ℹ️ No composition table provided. Skipping Fig.5(E)-style composition plot. "
          "To enable, set COMPOSITION_PATH or COMPOSITION_SHEET and include AM, MBA, APS (+ optional gelation class).")

# ---------- (Optional) Normalized tri-property cube (like paper’s normalized targets) ----------
# If you want a normalized 0–1 cube view for any 3 properties (easier to see “balanced” region):
NORM_AXES = [PROPERTY_X, PROPERTY_Y, PROPERTY_Z]
scaler = MinMaxScaler()
norm_vals = scaler.fit_transform(props_df[NORM_AXES].values)
norm_df = pd.DataFrame(norm_vals, columns=[f"norm_{c}" for c in NORM_AXES])
norm_df = pd.concat([props_df[["label","mat_type","mat_level"]].reset_index(drop=True), norm_df], axis=1)

fig_norm = px.scatter_3d(
    norm_df, x=f"norm_{PROPERTY_X}", y=f"norm_{PROPERTY_Y}", z=f"norm_{PROPERTY_Z}",
    color="mat_type",
    hover_data=["label","mat_level"]
)
fig_norm.update_traces(marker=dict(size=6, opacity=0.8))
fig_norm.update_layout(
    title=f"Normalized 3D Property Cube: {PROPERTY_X}, {PROPERTY_Y}, {PROPERTY_Z} (0–1)"
)
fig_norm.show()

print("\nDone. Tip: Change PROPERTY_X/PROPERTY_Y/PROPERTY_Z to any derived columns to plot different 3D views.")


Mounted at /content/drive
Derived property table (head):


/tmp/ipython-input-36696654.py:94: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(f(xs), xs))  # stress(kPa) × strain(fraction) -> kJ/m^3 (since kPa = kJ/m^3)


,label,mat_type,mat_level,UTS_kPa,Strain_UTS_frac,Fracture_strain_frac,Fracture_stress_kPa,Toughness_kJ_m3,E0_5_kPa,Yield_stress_kPa,Yield_strain_frac,Stress@5%_kPa,Stress@10%_kPa,Stress@15%_kPa,Stress@20%_kPa
0,Cel10_e,Cel,10.0,85.0,0.17,0.17,85.0,6.395003,440.0,30.800000,0.072000,22.0,42.0,67.0,NaN
1,Cel10_f,Cel,10.0,55.0,0.15,0.15,55.0,4.225000,440.0,23.271111,0.054889,22.0,35.0,55.0,NaN
2,Cel1_e,Cel,1.0,90.0,0.18,0.18,90.0,7.650005,500.0,29.000000,0.060000,25.0,45.0,70.0,NaN
3,Cel1_f,Cel,1.0,65.0,0.16,0.16,65.0,4.425005,400.0,21.200000,0.055000,20.0,32.0,50.0,NaN
4,Cel20_e,Cel,20.0,95.0,0.15,0.15,95.0,6.525000,560.0,NaN,NaN,28.0,55.0,95.0,NaN


ℹ️ No composition table provided. Skipping Fig.5(E)-style composition plot. To enable, set COMPOSITION_PATH or COMPOSITION_SHEET and include AM, MBA, APS (+ optional gelation class).



Done. Tip: Change PROPERTY_X/PROPERTY_Y/PROPERTY_Z to any derived columns to plot different 3D views.


ব্যবহার টিপস

3D property প্লটের অক্ষ বদলাতে PROPERTY_X/Y/Z-এ অন্য property নাম দিন (যেমন E0_5_kPa, Yield_stress_kPa, Stress@10%_kPa ইত্যাদি)।

Figure 5(E)-স্টাইল composition প্লট পেতে আলাদা composition টেবিলে AM, MBA, APS (এবং চাইলে gel_ok/gelation ক্লাস) রাখুন; COMPOSITION_PATH বা COMPOSITION_SHEET সেট করুন।

চাইলে LIKELY_*_COLS তালিকায় নিজের কলাম নাম যোগ করে ম্যাচিং সহজ করুন।

এই কোডটা কনসেপ্টually Figure 5(E)-এর 3D স্ক্যাটারের আদলেই বানানো—একদিকে composition 3D scatter (জেলেশন ক্লাস রঙে), আরেকদিকে আপনার dataset-এর multi-property 3D ভিউ—যেন skin sensor অ্যাপ্লিকেশনের দিকে properties jointly দেখা যায়।

চলুন মিটিং-এ আলোচনা করা 3D ফিগারগুলো—Figure-5(E) টাইপ composition 3D, সাথে property–property–property 3D, আর Bayesian (GP) surrogate থেকে prediction & EI (Expected Improvement)—আপনার Excel ডেটা থেকেই বানাই। নিচের একটা মাত্র Colab সেল কপি-পেস্ট করলেই চলবে। (রেফারেন্স হিসেবে যে বোয়েলার-প্লেট কাজ পেপারে—Bayesian অপ্টিমাইজেশন/লো-ডেটাসেট ML—ব্যবহৃত, সেটিই ফলো করা হয়েছে।
Paper link: https://drive.google.com/file/d/1PHsJty9wXimV0bX6MvlkcHj5SgdH33-2/view

In [26]:
# ==============================================================
# One-cell Colab: 3D figures from your PAAm hydrogel dataset
#  - Derive properties from stress–strain
#  - 3D property scatter (property-property-property)
#  - 3D composition scatter (AM–MBA–APS), if composition is available
#  - GP surrogate over AM–MBA–APS for a chosen property + isosurfaces
#  - Expected Improvement (EI) isosurface for Bayesian-style exploration
# ==============================================================

!pip -q install numpy pandas scipy scikit-learn plotly

# ---- Mount Google Drive (for your given path) ----
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from typing import Dict, Tuple, Optional, List
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.exceptions import ConvergenceWarning
import warnings

# ---------------- USER CONFIG ----------------
# Your stress–strain Excel file (paired columns: Strain(%)_<label>, Stress(kPa)_<label>)
DATA_PATH = "/content/drive/MyDrive/AI Training/paam_hydrogel_stress_strain_data_v2.xlsx"

# (Optional) Composition source (needed for AM–MBA–APS & GP):
#   Option A: a separate file (.xlsx/.csv)
COMPOSITION_PATH = ""   # e.g., "/content/drive/MyDrive/AI Training/composition_table.xlsx"
#   Option B: a sheet inside DATA_PATH
COMPOSITION_SHEET = ""  # e.g., "composition"

# Property 3D axes (change if you want other properties)
PROPERTY_X = "UTS_kPa"
PROPERTY_Y = "Fracture_strain_frac"   # unitless fraction
PROPERTY_Z = "Toughness_kJ_m3"

# Surrogate (GP) target property over composition (AM, MBA, APS)
SURROGATE_TARGET = "UTS_kPa"  # e.g., "UTS_kPa", "Toughness_kJ_m3", "E0_5_kPa", "Stress@10%_kPa", ...

# For the GP/EI grid (bigger -> heavier). 16–20 works well.
GRID_N = 18
EI_XI = 0.01   # EI exploration parameter
# --------------------------------------------

# ================= Utility: stress–strain -> properties =================
def find_pairs(df: pd.DataFrame) -> Tuple[Dict[str, str], Dict[str, str], list]:
    strain_cols = [c for c in df.columns if c.lower().startswith("strain")]
    stress_cols = [c for c in df.columns if c.lower().startswith("stress")]
    def lab(c): return c.split("_", 1)[1] if "_" in c else None
    labels_strain = {lab(c): c for c in strain_cols if lab(c) is not None}
    labels_stress = {lab(c): c for c in stress_cols if lab(c) is not None}
    labels = sorted(set(labels_strain).intersection(labels_stress))
    return labels_strain, labels_stress, labels

def to_fraction(eps_raw: np.ndarray) -> np.ndarray:
    # e.g., 0, 500, 1000 -> 0.0, 0.05, 0.10 (fraction)
    return (eps_raw.astype(float) / 100.0) / 100.0

def ensure_sorted(eps: np.ndarray, sig: np.ndarray):
    idx = np.argsort(eps)
    return eps[idx], sig[idx]

def interp_curve(eps: np.ndarray, sig: np.ndarray):
    x = np.asarray(eps, float); y = np.asarray(sig, float)
    lo, hi = float(x.min()), float(x.max())
    def f(xq):
        xq = np.asarray(xq, float)
        return np.interp(np.clip(xq, lo, hi), x, y)
    return f, (lo, hi)

def linear_fit_window(eps: np.ndarray, sig: np.ndarray, a: float, b: float):
    f, (lo, hi) = interp_curve(eps, sig)
    a_eff, b_eff = max(lo, a), min(hi, b)
    if b_eff - a_eff < 0.01:  # need at least ~1% strain span
        return None
    xs = np.linspace(a_eff, b_eff, 20); ys = f(xs)
    X = np.vstack([xs, np.ones_like(xs)]).T
    slope, intercept = np.linalg.lstsq(X, ys, rcond=None)[0]
    return float(slope), float(intercept)

def yield_offset(eps: np.ndarray, sig: np.ndarray, E_init: Optional[float], offset: float = 0.002):
    if E_init is None or not np.isfinite(E_init): return None, None
    f, (lo, hi) = interp_curve(eps, sig)
    xs = np.linspace(lo, hi, 600)
    g = f(xs) - E_init*(xs - offset)
    s = np.sign(g); idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0: return None, None
    i = idx[0]; x0, x1 = xs[i], xs[i+1]; y0, y1 = g[i], g[i+1]
    eps_y = x0 if (y1 - y0) == 0 else x0 - y0*(x1 - x0)/(y1 - y0)
    return float(eps_y), float(f(eps_y))

def integrate_toughness(eps: np.ndarray, sig: np.ndarray, up_to: Optional[float] = None) -> float:
    f, (lo, hi) = interp_curve(eps, sig)
    b = hi if up_to is None else max(lo, min(hi, up_to))
    xs = np.linspace(lo, b, 400)
    return float(np.trapz(f(xs), xs))  # kPa × strain -> kJ/m^3

def stress_at(f, p: float, lo: float, hi: float):
    x = p/100.0
    return float(f(x)) if lo <= x <= hi else np.nan

import re
def parse_label_recipe(label: str):
    m = re.match(r"([A-Za-z]+)(\d+)?", label)
    mat_type = m.group(1) if m else None
    mat_level = float(m.group(2)) if (m and m.group(2)) else 0.0
    return mat_type, mat_level

def derive_properties_table(stress_strain_path: str) -> pd.DataFrame:
    df = pd.read_excel(stress_strain_path)
    labels_strain, labels_stress, labels = find_pairs(df)
    rows = []
    for lab in labels:
        eps_raw = df[labels_strain[lab]].to_numpy(dtype=float)
        sig_raw = df[labels_stress[lab]].to_numpy(dtype=float)
        m = np.isfinite(eps_raw) & np.isfinite(sig_raw)
        eps_raw, sig_raw = eps_raw[m], sig_raw[m]
        if len(eps_raw) < 3:
            continue
        eps = to_fraction(eps_raw)
        eps, sig = ensure_sorted(eps, sig_raw)
        f, (lo, hi) = interp_curve(eps, sig)

        # Initial modulus over [0, min(0.05, hi)] with min 0.01 span
        fit = linear_fit_window(eps, sig, 0.00, min(0.05, hi))
        E0_5 = fit[0] if fit is not None else np.nan

        # Yield via 0.2% offset
        eps_y, sig_y = yield_offset(eps, sig, E0_5, offset=0.002)

        # Main properties
        uts = float(sig.max()); i_uts = int(sig.argmax()); strain_uts = float(eps[i_uts])
        frac_strain = float(eps.max()); frac_stress = float(sig[-1])
        toughness = integrate_toughness(eps, sig, None)
        s5  = stress_at(f, 5, lo, hi); s10 = stress_at(f, 10, lo, hi)
        s15 = stress_at(f, 15, lo, hi); s20 = stress_at(f, 20, lo, hi)

        mat_type, mat_level = parse_label_recipe(lab)
        rows.append({
            "label": lab, "mat_type": mat_type, "mat_level": mat_level,
            "E0_5_kPa": E0_5,
            "Yield_strain_frac": eps_y if eps_y is not None else np.nan,
            "Yield_stress_kPa":  sig_y if sig_y is not None else np.nan,
            "UTS_kPa": uts, "Strain_UTS_frac": strain_uts,
            "Fracture_strain_frac": frac_strain, "Fracture_stress_kPa": frac_stress,
            "Toughness_kJ_m3": toughness,
            "Stress@5%_kPa": s5, "Stress@10%_kPa": s10, "Stress@15%_kPa": s15, "Stress@20%_kPa": s20
        })
    return pd.DataFrame(rows)

# ---------- Derive properties ----------
props_df = derive_properties_table(DATA_PATH)
if props_df.empty:
    raise RuntimeError("No valid stress–strain pairs found. Check column names like Strain(%)_<label> and Stress(kPa)_<label>.")

print("Derived properties (head):")
display(props_df.head())

# ================== Load composition (if available) ==================
def load_composition() -> Optional[pd.DataFrame]:
    comp_df = None
    if COMPOSITION_PATH and Path(COMPOSITION_PATH).exists():
        comp_df = pd.read_csv(COMPOSITION_PATH) if COMPOSITION_PATH.lower().endswith(".csv") else pd.read_excel(COMPOSITION_PATH)
    elif COMPOSITION_SHEET:
        comp_df = pd.read_excel(DATA_PATH, sheet_name=COMPOSITION_SHEET)
    return comp_df

comp_df = load_composition()

LIKELY_AM_COLS  = ["am", "aam", "acrylamide", "am_wt", "aam_wt"]
LIKELY_MBA_COLS = ["mba", "bis", "n,n-methylenebisacrylamide", "bis_wt", "mba_wt"]
LIKELY_APS_COLS = ["aps", "ammonium persulfate", "aps_wt"]
LIKELY_CLASS_COLS = ["gel_ok", "gelation", "class", "is_gel", "formed", "status"]

def pick_first_match(cols, candidates):
    cl = {c.lower(): c for c in cols}
    # direct match
    for name in candidates:
        if name in cl:
            return cl[name]
    # startswith fuzzy
    for name in candidates:
        for lc, orig in cl.items():
            if lc.startswith(name):
                return orig
    return None

merged = None
am_col = mba_col = aps_col = cls_col = None

if comp_df is not None:
    if "label" in comp_df.columns and "label" in props_df.columns:
        merged = props_df.merge(comp_df, on="label", how="inner")
    else:
        # fallback: if no label, assume composition table aligns by row order with props_df (not recommended)
        merged = pd.concat([props_df.reset_index(drop=True), comp_df.reset_index(drop=True)], axis=1)

    lower_cols = [c.lower() for c in merged.columns]
    col_map = {c.lower(): c for c in merged.columns}
    am_col  = pick_first_match(col_map.keys(), LIKELY_AM_COLS)
    mba_col = pick_first_match(col_map.keys(), LIKELY_MBA_COLS)
    aps_col = pick_first_match(col_map.keys(), LIKELY_APS_COLS)
    cls_col = pick_first_match(col_map.keys(), LIKELY_CLASS_COLS)

# ================== 3D Property scatter ==================
for col in [PROPERTY_X, PROPERTY_Y, PROPERTY_Z]:
    if col not in props_df.columns:
        raise RuntimeError(f"Property axis '{col}' not found. Available: {list(props_df.columns)}")

fig_prop = px.scatter_3d(
    props_df, x=PROPERTY_X, y=PROPERTY_Y, z=PROPERTY_Z,
    color="mat_type", symbol="mat_type",
    hover_data=["label","mat_level","E0_5_kPa","Yield_stress_kPa","Stress@10%_kPa","Toughness_kJ_m3"]
)
fig_prop.update_traces(marker=dict(size=6, opacity=0.85))
fig_prop.update_layout(
    title=f"3D Property Space: {PROPERTY_X} vs {PROPERTY_Y} vs {PROPERTY_Z}",
    legend_title="mat_type"
)
fig_prop.show()

# ================== 3D Composition scatter (AM–MBA–APS) ==================
if merged is not None and am_col and mba_col and aps_col:
    # Decide size/color: use surrogate target if present
    size_col = SURROGATE_TARGET if SURROGATE_TARGET in merged.columns else None
    color_col = None
    if cls_col and cls_col in merged.columns:
        # Normalize class to readable text
        v = merged[cls_col].astype(str).str.lower()
        cls = v.map(lambda s: "Gel formed" if s in ["1","true","yes","y","ok","formed","gel","good"] else ("Cutoff" if s in ["0","false","no","n","cutoff","bad","fail"] else "Unknown"))
        merged["_gel_class_"] = cls
        color_col = "_gel_class_"

    fig_comp = px.scatter_3d(
        merged, x=am_col, y=mba_col, z=aps_col,
        color=color_col if color_col else None,
        symbol=color_col if color_col else None,
        size=size_col if size_col else None,
        hover_data=["label"] if "label" in merged.columns else None
    )
    fig_comp.update_traces(marker=dict(size=5, opacity=0.9))
    ttl = f"3D Composition: {am_col} vs {mba_col} vs {aps_col}"
    if color_col:
        ttl += " — gelation class"
    if size_col:
        ttl += f" (size ~ {size_col})"
    fig_comp.update_layout(title=ttl)
    fig_comp.show()
else:
    print("ℹ️ Composition 3D scatter skipped: AM/MBA/APS or composition table not found. Set COMPOSITION_PATH/COMPOSITION_SHEET and ensure columns exist.")

# ---------- (Optional) Normalized tri-property cube (like paper’s normalized targets) ----------
# If you want a normalized 0–1 cube view for any 3 properties (easier to see “balanced” region):
NORM_AXES = [PROPERTY_X, PROPERTY_Y, PROPERTY_Z]
scaler = MinMaxScaler()
norm_vals = scaler.fit_transform(props_df[NORM_AXES].values)
norm_df = pd.DataFrame(norm_vals, columns=[f"norm_{c}" for c in NORM_AXES])
norm_df = pd.concat([props_df[["label","mat_type","mat_level"]].reset_index(drop=True), norm_df], axis=1)

fig_norm = px.scatter_3d(
    norm_df, x=f"norm_{PROPERTY_X}", y=f"norm_{PROPERTY_Y}", z=f"norm_{PROPERTY_Z}",
    color="mat_type",
    hover_data=["label","mat_level"]
)
fig_norm.update_traces(marker=dict(size=6, opacity=0.8))
fig_norm.update_layout(
    title=f"Normalized 3D Property Cube: {PROPERTY_X}, {PROPERTY_Y}, {PROPERTY_Z} (0–1)"
)
fig_norm.show()

# ================== GP surrogate over composition + Isosurface ==================
def make_gp_isosurfaces(df: pd.DataFrame, x_col: str, y_col: str, z_col: str, y_target_col: str,
                        grid_n:int=18, title_prefix:str=""):
    # drop NA
    use = df[[x_col, y_col, z_col, y_target_col]].dropna()
    if len(use) < 8:
        print("⚠️ Not enough rows for GP (~>=8 recommended). Skipping GP/EI.")
        return

    X = use[[x_col, y_col, z_col]].values.astype(float)
    y = use[y_target_col].values.astype(float)

    # Scale inputs to [0,1] for stable GP
    xsc = MinMaxScaler()
    Xn = xsc.fit_transform(X)

    # Simple, robust kernel
    kernel = ConstantKernel(1.0, (1e-2, 1e3)) * Matern(length_scale=np.ones(3), length_scale_bounds=(1e-2, 10.0), nu=2.5) + WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-6, 1e-1))
    gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True, n_restarts_optimizer=2, random_state=42)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        gp.fit(Xn, y)

    # Build grid in normalized space
    ax = np.linspace(0, 1, grid_n)
    AY, BY, CY = np.meshgrid(ax, ax, ax, indexing="ij")  # shape (n,n,n)
    Xgrid = np.stack([AY.ravel(), BY.ravel(), CY.ravel()], axis=1)
    mu, std = gp.predict(Xgrid, return_std=True)

    # Expected Improvement (maximize y)
    y_best = np.max(y)
    eps = 1e-12
    improv = mu - y_best - EI_XI
    Z = improv / (std + eps)
    from math import sqrt, pi, exp
    # Normal pdf & cdf
    pdf = 1.0/np.sqrt(2*np.pi) * np.exp(-0.5*Z**2)
    from scipy.stats import norm
    cdf = norm.cdf(Z)
    EI = np.maximum(0.0, improv)*cdf + std*pdf

    # Back-transform grid to original units for plotting axes
    # sample a unit cube grid and inverse_transform
    # Build arrays corresponding to the 3 axes (vector form)
    def inv_axis(idx):
        # unit cube points for a single axis across all combinations -> take unique
        U = np.zeros((grid_n, 3))
        U[:, idx] = ax
        inv = xsc.inverse_transform(U)
        return inv[:, idx]

    gx = inv_axis(0); gy = inv_axis(1); gz = inv_axis(2)

    # -------- Prediction isosurface --------
    vol_mu = mu.reshape(grid_n, grid_n, grid_n)
    # Choose a few iso levels: quartiles of mu
    qs = np.quantile(mu[np.isfinite(mu)], [0.25, 0.5, 0.75])
    fig_mu = go.Figure()
    for q in qs:
        fig_mu.add_trace(go.Isosurface(
            x=np.repeat(gx, grid_n*grid_n),
            y=np.tile(np.repeat(gy, grid_n), grid_n),
            z=np.tile(gz, grid_n*grid_n),
            value=mu,
            isomin=q, isomax=q,
            surface_count=1,
            showscale=False,
            opacity=0.15,
            caps=dict(x_show=False, y_show=False, z_show=False),
            name=f"μ≈{q:.2f}"
        ))
    # overlay observed points colored by y
    fig_mu.add_trace(go.Scatter3d(
        x=use[x_col], y=use[y_col], z=use[z_col],
        mode='markers',
        marker=dict(size=5, color=use[y_target_col], colorscale='Viridis', showscale=True, colorbar=dict(title=y_target_col)),
        name='observed'
    ))
    fig_mu.update_layout(
        title=f"{title_prefix} GP μ (prediction) isosurfaces + observed points",
        scene=dict(xaxis_title=x_col, yaxis_title=y_col, zaxis_title=z_col)
    )
    fig_mu.show()

    # -------- EI isosurface --------
    vol_ei = EI.reshape(grid_n, grid_n, grid_n)
    # High-EI cutoff at, say, top 10–20% EI
    thr = np.quantile(EI[np.isfinite(EI)], 0.80)
    fig_ei = go.Figure(go.Isosurface(
        x=np.repeat(gx, grid_n*grid_n),
        y=np.tile(np.repeat(gy, grid_n), grid_n),
        z=np.tile(gz, grid_n*grid_n),
        value=EI,
        isomin=thr, isomax=EI.max(),
        surface_count=2,
        colorscale='Plasma',
        opacity=0.35,
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorbar=dict(title="EI")
    ))
    fig_ei.add_trace(go.Scatter3d(
        x=use[x_col], y=use[y_col], z=use[z_col],
        mode='markers',
        marker=dict(size=4, color='black'),
        name='observed'
    ))
    fig_ei.update_layout(
        title=f"{title_prefix} Expected Improvement (EI) — high-EI region",
        scene=dict(xaxis_title=x_col, yaxis_title=y_col, zaxis_title=z_col)
    )
    fig_ei.show()

# Run GP only if composition present
if merged is not None and am_col and mba_col and aps_col:
    if SURROGATE_TARGET in merged.columns:
        make_gp_isosurfaces(merged, am_col, mba_col, aps_col, SURROGATE_TARGET, grid_n=GRID_N,
                            title_prefix=f"{SURROGATE_TARGET}: ")
    else:
        print(f"ℹ️ Surrogate target '{SURROGATE_TARGET}' not found in merged table. Available: {list(merged.columns)}")

print("\nDone. Tips:")
print("• PROPERTY_X/Y/Z বদলে যে কোন ৩টি derived property প্লট করুন।")
print("• COMPOSITION_PATH/COMPOSITION_SHEET দিন এবং AM/MBA/APS কলাম ঠিক থাকলে composition 3D ও GP/EI প্লট আসবে।")

Mounted at /content/drive
Derived properties (head):


/tmp/ipython-input-3196232463.py:102: DeprecationWarning:

`trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.



,label,mat_type,mat_level,E0_5_kPa,Yield_strain_frac,Yield_stress_kPa,UTS_kPa,Strain_UTS_frac,Fracture_strain_frac,Fracture_stress_kPa,Toughness_kJ_m3,Stress@5%_kPa,Stress@10%_kPa,Stress@15%_kPa,Stress@20%_kPa
0,Cel10_e,Cel,10.0,440.0,0.072000,30.800000,85.0,0.17,0.17,85.0,6.395003,22.0,42.0,67.0,NaN
1,Cel10_f,Cel,10.0,440.0,0.054889,23.271111,55.0,0.15,0.15,55.0,4.225000,22.0,35.0,55.0,NaN
2,Cel1_e,Cel,1.0,500.0,0.060000,29.000000,90.0,0.18,0.18,90.0,7.650005,25.0,45.0,70.0,NaN
3,Cel1_f,Cel,1.0,400.0,0.055000,21.200000,65.0,0.16,0.16,65.0,4.425005,20.0,32.0,50.0,NaN
4,Cel20_e,Cel,20.0,560.0,NaN,NaN,95.0,0.15,0.15,95.0,6.525000,28.0,55.0,95.0,NaN


ℹ️ Composition 3D scatter skipped: AM/MBA/APS or composition table not found. Set COMPOSITION_PATH/COMPOSITION_SHEET and ensure columns exist.



Done. Tips:
• PROPERTY_X/Y/Z বদলে যে কোন ৩টি derived property প্লট করুন।
• COMPOSITION_PATH/COMPOSITION_SHEET দিন এবং AM/MBA/APS কলাম ঠিক থাকলে composition 3D ও GP/EI প্লট আসবে।


ব্যবহার টিপস

Property 3D: PROPERTY_X/PROPERTY_Y/PROPERTY_Z–তে আপনাদের আগ্রহের ৩টি property (যেমন E0_5_kPa, Yield_stress_kPa, Stress@10%_kPa) বসিয়ে দিন।

Composition 3D & GP/EI: composition টেবিল দিন (COMPOSITION_PATH বা COMPOSITION_SHEET), আর কলাম নাম AM/MBA/APS-এর সাথে ম্যাচ করলে 3D composition scatter, GP prediction isosurfaces, আর EI isosurface দেখাবে—পেপারে বর্ণিত BO-স্টাইল অনুসন্ধানের (exploration) ভিজুয়াল গাইড হিসেবে উপকারী।

summerstudent2022_PAAmMachineLe…

গ্রিড সাইজ/সময়: GRID_N বড় হলে isosurface আরও স্মুথ, তবে কম্পিউট ভারী হয়। 16–20 ভাল ব্যালান্স।

Target change: SURROGATE_TARGET বদলে Toughness/UTS/অন্যান্য property’র জন্যও GP/EI প্লট করতে পারবেন।

চাইলে আপনার composition ফাইল/কলাম নাম একবার বললে, এগুলো আমি প্রিসেট করে দেব, যাতে আর কিছু না বদলাতে হয়।